In [2]:
import numpy as np
import scipy as sp
import math
import functools as ft
from qiskit.quantum_info import Statevector, SparsePauliOp, Operator
from matplotlib import pyplot as plt

#### Constants

In [3]:
# Can't use m and g too big because too massive and too much coupling means the particles are not going to move. 
# It's not just their ratio that matters but also the numerical values because of the kinetic term

#m = 0.5
#g = math.sqrt(m*16/3)

g = 2.
m = 1.

In [4]:
g

1.632993161855452

#### Helper Functions

In [7]:
# Define the Pauli Matrices

x = np.matrix([[0.,1.],[1.,0.]])
y = np.matrix([[0.,-1j],[1j,0.]])
z = np.matrix([[1.,0.],[0.,-1.]])
id = sp.sparse.identity(2)

proj0 = (z + id)/2.
proj1 = (id - z)/2.

In [8]:
def ry(angle):
    ang = angle/2
    return np.matrix([[math.cos(ang),-math.sin(ang)],[math.sin(ang),math.cos(ang)]])

In [9]:
# Pad operator on one qubit with identities to expand it to dimension of the full hilbert space
# "op" is the 1 qubit operator. For example Pauli x or Projection
# "site" is the site where the operator is to be placed
# "total" is the total dimension of the full hilbert space

def pad_op(site,total,op):
    return ft.reduce(sp.sparse.kron, [sp.sparse.identity(2**(site)),op,sp.sparse.identity(2**(total-site-1))])

In [ ]:
def overlap(state1,state2):
    return np.abs(state1.conj().T @ state2)

# Construct Hamiltonian for Scipy calculations

In [10]:
# Calculate the electric energy at each site

def He_persite_scipy(nq,g,site):
    return 3 * g**2/8 * pad_op(site,nq,proj1)

In [ ]:
# Plot the electric energy at each site for all the sites

def plot_E_per_site(state,nq):      # state is a csr or csc array
    E_exps = []
    for loc in range(nq):
        E_exps += [(np.real(state.conj().T @ He_persite_scipy(nq,g,loc) @ state)).toarray()[0][0]] # Should be CT(state).H.state
    plt.plot(list(range(nq)), E_exps,'b.',linestyle = "-", label = 'Electric Hamiltonian Expectation Value')
    return E_exps

# Construct the Hamiltonian for Qiskit VQE

#### Helper Functions

In [38]:
# Generate Pauli operators
# "op" is the Pauli operator. For example Pauli x or Pauli z
# "loc" is the site where the operator is to be placed

def pauli_op(op,loc,nq,cons = 1):
    operator = SparsePauliOp.from_sparse_list([(op, [loc], cons)], num_qubits = nq)
    return operator

In [39]:
# Generate projection operators P0 or P1 with dimension of the full hilbert space. 
# "which" is 0 for P0 and 1 for P1. 
# "loc" is the site where the projection is to be placed.

def projection(which, loc, nq):
    if which == 0:
        operator = SparsePauliOp.from_sparse_list([("Z", [loc], 1), ("I", [loc], 1)], num_qubits=nq)/2
    elif which == 1:
        operator = SparsePauliOp.from_sparse_list([("Z", [loc], -1), ("I", [loc], 1)], num_qubits=nq)/2
    else:
        raise Exception("Type of projection operator not supported. Choose 0 or 1.")
    return operator

#### Kinetic Hamiltonian

In [40]:
def Hkin(nq):
    
    # Initialize "0" operators
    op1 = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)
    op2 = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)

    for i in range(nq):
        op1 += projection(0,i,nq).dot(pauli_op("X",(i+1)%nq,nq)).dot(projection(0,(i+2)%nq,nq))
        op2 += projection(1,i,nq).dot(pauli_op("X",(i+1)%nq,nq)).dot(projection(1,(i+2)%nq,nq))
        
    return (math.sqrt(2)*op1/2 + op2/math.sqrt(2)/2).simplify()

#### Mass Hamiltonian

In [41]:
def Hm(nq,m):
    op = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)

    for i in range(nq):
        op += projection(0,i,nq).dot(projection(1,(i+1)%nq,nq)) + projection(1,i,nq).dot(projection(0,(i+1)%nq,nq))
        
    return (m*op).simplify()

#### Electric Hamiltonian

In [42]:
def He(nq,g):
    op = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits = nq)
    
    for i in range(nq):
        op += projection(1,i,nq) 
        
    return (3*g**2/8*op).simplify()

In [43]:
def He_persite(site,nq = nQ,g = g):
    return 3 * g**2/8 * projection(1,site,nq) 

#### Full Hamiltonian with mass m and gauge coupling g

In [44]:
def Hfull(nq,m,g):
    return (Hkin(nq)+He(nq,g)+Hm(nq,m)).simplify()

#### Result Handling etc..

In [ ]:
def plotAmpDict(AmpDict,isprob = False):
    if isprob == True:
        plt.plot(list(range(2**nQ)), np.sqrt(list(AmpDict.values())),label = 'QC Simulator')
    else: 
        plt.plot(list(range(2**nQ)), AmpDict.values(),label = 'QC Simulator')
    plt.xlabel("State", fontsize=16)
    plt.ylabel("Amplitude", fontsize=16)
    plt.legend()
    plt.show() 

In [ ]:
# This function handles results from the sampler by padding the locations with no values 0.
# Generates a list of probabilities in order of state 0-31
# Could also want amplitude here instead of the probability
# keys of the dictionary are the state numbers 0-2**nQ. Would generate all 0s if the keys are bit strings. 

def padDict(dicta):
    ordered_prob = []
    for i in range(2**nQ):
        if i in dicta.keys(): 
            ordered_prob.append(dicta[i])
        else: ordered_prob.append(0)
    return ordered_prob 

In [ ]:
def index_from_bitstring(bitstring):
    bitstring_reversed = reversed(bitstring)
    index = 0
    for i,c in enumerate(bitstring_reversed):
        if c == '1':
            index += np.power(2, i)
    return index

# works when keys are bitstrings
def padbitstringDict(dictaa,nq = nQ):
    ordered_prob = [0]*2**nq
    for item in dictaa.keys():
        ordered_prob[index_from_bitstring(item)] = dictaa[item]
    return ordered_prob 

# Results for comparision

In [ ]:
mathematica_result_6_SU2 = [0.909337419761728, -0.1656454077194386, -0.16564540771943856, \
3.589682898572368e-17, -0.16564540771943853, 0.03073593140751702, \
-3.1114743909916993e-17, -0.001522549560842467, -0.16564540771943848, \
0.03013800986744891, 0.030735931407517028, 1.1036715171105871e-17, \
-1.9087092292274227e-17, -4.628992161664347e-18, \
-0.0015225495608425068, 2.012819470989394e-18, -0.16564540771943864, \
0.030735931407517056, 0.030138009867448943, -2.2030597050776493e-17, \
0.03073593140751706, -0.005811352790172923, -3.37331819017979e-18, \
0.00029199166995784204, 3.13629642143285e-18, \
-2.1456710680372885e-17, 2.5110118953031286e-18, \
1.3321887191664034e-6, -0.0015225495608425191, 0.0002919916699578692, \
-7.883589762993288e-18, -0.000020296026949688266, \
-0.16564540771943853, 4.16759149078233e-18, 0.030735931407517042, \
-0.0015225495608425202, 0.03013800986744892, -6.977070614877761e-18, \
-9.942312466179429e-18, 1.9401218429378626e-17, 0.030735931407517042, \
-5.6922593764805414e-18, -0.0058113527901729155, \
0.0002919916699578426, 7.213369338939558e-18, 1.3321887191551184e-6, \
0.0002919916699578479, -0.00002029602694969559, \
1.1535482600692247e-17, -0.001522549560842509, \
-2.0729308934489092e-17, 2.450403383076433e-17, \
2.4596544125266465e-17, 0.0002919916699578683, 1.332188719178406e-6, \
-0.000020296026949678566, -0.0015225495608425055, \
5.237212966581615e-18, 0.0002919916699578449, \
-0.000020296026949682252, -1.9134033403807665e-17, \
-0.000020296026949661934, -0.000020296026949685234, \
4.4055123216714275e-6]

In [ ]:
mathematica_result_8_SU2 = [-0.9028789064865107, 0.14703178438136397, 0.14703178438136394, \
1.672893829997504e-22, 0.14703178438136402, -0.02429754776917324, \
-2.330439202029597e-27, 0.0010022464240444207, 0.147031784381364, \
-0.02393463397483345, -0.02429754776917323, -1.5418811678368127e-23, \
4.392152521550533e-22, -3.717294932930091e-23, 0.0010022464240444138, \
2.0833778203591884e-24, 0.1470317843813639, -0.02394444811329397, \
-0.023934633974833436, -5.525908291714791e-22, -0.02429754776917321, \
0.004015665189291175, -1.3474933889111933e-27, \
-0.00016651367356683035, 7.094348868570456e-24, \
-1.3539395240289562e-24, -8.183245413668051e-25, \
-2.5424687271479144e-7, 0.0010022464240444012, \
-0.00016651367356682807, 3.8713177377969333e-26, \
9.204859259226372e-6, 0.147031784381364, -0.023934633974833457, \
-0.02394444811329397, -1.0918513169960404e-23, -0.023934633974833457, \
0.003953755197580843, 3.842883076699491e-27, -0.00016303112999586073, \
-0.024297547769173245, 0.003953755197580837, 0.004015665189291174, \
1.3461521233122803e-24, -2.353350682625907e-21, \
1.306929925420424e-22, -0.00016651367356682978, \
-3.666600157897346e-24, 1.0778941694738805e-20, \
-8.2573033165409805e-22, -3.701240216302443e-21, \
8.561755269564074e-196, -4.0448397821862524e-20, \
1.9743951541362224e-21, -2.5424687270628045e-7, \
-4.7221929913138885e-23, 0.0010022464240444342, \
-0.00016303112999585992, -0.00016651367356683325, \
-4.71139845368735e-25, 1.4363445755316256e-20, \
-7.309935307402434e-22, 9.204859259214849e-6, 1.8537618785127546e-23, \
0.14703178438136394, -0.024297547769173217, -0.023934633974833464, \
-1.0585109077746277e-21, -0.02394444811329397, 0.004015665189291166, \
-2.1977047425426526e-28, -0.0001665136735668284, \
-0.023934633974833464, 0.0039537551975808456, 0.003953755197580844, \
6.846505027383871e-23, -2.864362687876259e-23, \
3.1832018996824425e-24, -0.00016303112999586247, \
-2.2151179087650944e-24, -0.024297547769173217, \
0.0040156651892911755, 0.003953755197580842, 8.367430384329658e-21, \
0.004015665189291164, -0.0006735765728137208, 5.724776045926116e-27, \
0.00002808029335463949, -6.680439975774064e-23, \
1.3817135399101605e-23, 5.4644065301502285e-24, 4.351109543327875e-8, \
-0.00016651367356682558, 0.000028080293354615934, \
-1.8489166755394184e-25, -1.5627684549318315e-6, \
-1.248745672553658e-22, 9.06446803696193e-22, 8.871327671036783e-24, \
-2.5424687271435776e-7, 1.2131762138943279e-23, \
-6.249949809950369e-23, 0., 1.9316858925369715e-24, \
2.9483026906088996e-22, -5.422024809946692e-21, \
-1.2142356731269986e-23, 4.351109543511851e-8, \
-2.5424687271105094e-7, 4.351109542471017e-8, 2.3426542036412667e-25, \
-5.598066795221215e-9, 0.0010022464240444138, \
-0.00016651367356683024, -0.00016303112999585672, \
-1.125932410326661e-19, -0.00016651367356682558, \
0.00002808029335459639, -5.357963957266071e-26, \
-1.177204805633652e-6, 2.010825104136882e-21, -3.107585379459642e-20, \
-9.51869784146524e-23, -5.598066766845717e-9, 9.204859259213661e-6, \
-1.562768454928466e-6, 2.2555258732847185e-24, 9.045175801457224e-8, \
0.14703178438136397, -1.220011414050662e-22, -0.024297547769173224, \
0.0010022464240444034, -0.023934633974833464, 1.1882222113056048e-23, \
2.0534867095458682e-26, -5.117346295126266e-25, \
-0.023944448113293968, 3.1357378561237967e-22, 0.004015665189291169, \
-0.0001665136735668289, -1.879899408357281e-21, \
-2.5424687270411204e-7, -0.00016651367356682848, \
9.204859259221034e-6, -0.023934633974833436, 7.960100271970925e-24, \
0.003953755197580839, -0.0001630311299958583, 0.003953755197580839, \
-9.419505802350925e-25, -1.939001309950994e-27, \
4.6652124494637283e-26, -4.632716173895897e-25, \
-2.4545467326488633e-91, 6.084304176166708e-26, \
-1.4054961282916242e-27, -0.00016303112999585656, \
2.2355390728578402e-26, -3.1941457815916087e-27, \
-1.5261247747403697e-27, -0.024297547769173238, \
8.607387806038228e-22, 0.004015665189291157, -0.00016651367356682113, \
0.003953755197580844, -6.032098094217878e-23, -5.702257287615161e-26, \
1.9056685197201566e-24, 0.004015665189291171, -4.721120107243147e-21, \
-0.0006735765728137083, 0.000028080293354628694, \
2.859567601980114e-20, 4.351109543566061e-8, 0.000028080293354649917, \
-1.5627684549327995e-6, -6.052959470114969e-20, \
-2.542468727114304e-7, 4.638640809045109e-20, -2.285261686687121e-19, \
6.047871076070158e-19, 4.351109543164906e-8, 4.3511095424388294e-8, \
-5.598066779500284e-9, -0.00016651367356682685, \
-6.060258688943295e-20, 0.000028080293354610425, \
-1.1772048056356613e-6, -1.9971036691798466e-19, \
-5.598066788485503e-9, -1.5627684549269276e-6, 9.04517580148224e-8, \
1.8236081028816095e-20, 0.0010022464240444294, \
-1.3332041781782524e-21, 1.5490857649810416e-20, \
-7.831097928505045e-20, -0.00016651367356683067, \
-2.54246872711322e-7, 9.204859259212062e-6, -1.1895979174859056e-21, \
-0.00016303112999586003, 1.3716066392563935e-22, \
-7.393931704587157e-22, 0., -4.941295770810606e-25, \
-3.668684387516338e-24, 1.7670817199992778e-23, \
-9.766213315380859e-20, -0.00016651367356683588, \
-5.527441976259849e-21, -2.37319767678676e-19, \
1.1912705333076102e-18, 0.000028080293354629053, \
4.3511095420051486e-8, -1.5627684549305769e-6, \
-2.5424687272563346e-7, 3.06290383990086e-20, 4.351109543626031e-8, \
-5.598066779852649e-9, -3.664502290720722e-19, -1.177204805608563e-6, \
-5.598066775701976e-9, 9.045175801352877e-8, 0.0010022464240444363, \
-1.6781194450210804e-20, -0.00016651367356683024, \
9.204859259221467e-6, -0.0001630311299958635, 7.98611469039033e-22, \
4.521306146508207e-25, -1.902449148891214e-23, \
-0.0001665136735668337, 2.5750195955941447e-19, \
0.000028080293354612858, -1.5627684549386046e-6, \
-9.986609437995582e-19, -5.5980667803642176e-9, \
-1.1772048056158814e-6, 9.0451758015471e-8, 5.692061405548898e-19, \
9.20485925920791e-6, 0., 6.938893903907228e-18, 0., \
-1.5627684548658038e-6, -5.5980668989263904e-9, 9.045175786148718e-8, \
9.204859259231979e-6, 0., -1.5627684549369275e-6, \
9.045175801468495e-8, 2.855517471783697e-17, 9.045175788895053e-8, \
9.045175806321835e-8, -1.5119265573776187e-8]

In [ ]:
mathematica_result_12_SU2 = [0.8579149985347635, -0.13970951381782784, -0.13970951381782787, \
1.3797666863832662e-16, -0.13970951381782798, 0.023087514789948364, \
-6.543027190345749e-17, -0.0009523338813055846, -0.1397095138178278, \
0.022742687730454538, 0.023087514789948364, 1.6610042806461937e-17, \
-6.264527514368026e-17, -1.0165007935043763e-17, \
-0.0009523338813056842, 9.022377566344065e-18, -0.1397095138178282, \
0.022751689048796517, 0.022742687730454684, -4.885402735565188e-17, \
0.02308751478994841, -0.0038156294832041157, -1.8369410497959913e-18, \
0.00015821781141703113, 3.235651083098818e-17, \
-4.216162926363717e-17, 2.260421498693502e-18, 2.415652135985082e-7, \
-0.0009523338813057404, 0.00015821781141705623, \
-4.0421496561867624e-19, -8.746122967213346e-6, -0.13970951381782779, \
0.02275136494773184, 0.022751689048796486, -5.1654034953353397e-17, \
0.022742687730454597, -0.003758302156981505, -1.0267828875567544e-17, \
0.00015499869964182445, 0.02308751478994839, -0.0037583021569814992, \
-0.0038156294832041226, 1.087960222597832e-18, \
2.8321928463001157e-17, -8.768880740457734e-18, \
0.0001582178114170309, -3.575582816574168e-18, \
1.4692218043122718e-17, -2.528740141994176e-17, \
-3.53754376457792e-17, 2.6312160708059453e-17, 2.649749581844649e-17, \
6.0366278754588636e-18, 2.4156521362077664e-7, \
-1.4448234177643241e-18, -0.0009523338813057007, \
0.00015499869964183456, 0.00015821781141703496, \
-3.9083782900427985e-18, -4.401143610600702e-17, \
1.1465015130702836e-17, -8.746122967216126e-6, \
-2.5641481891392834e-18, -0.13970951381782815, 0.022751378955961035, \
0.02275136494773188, -5.094586397670241e-17, 0.022751689048796445, \
-0.003759801482243192, -4.676785235402043e-18, \
0.00015508867446749266, 0.022742687730454604, -0.0037021810510366815, \
-0.0037583021569815105, 1.3117689002334215e-18, \
2.1015804252571075e-17, -9.55075115548318e-18, \
0.00015499869964182264, 7.901853481345853e-19, 0.023087514789948353, \
-0.0037598014822431665, -0.003758302156981483, \
1.2037005490736287e-17, -0.0038156294832040996, \
0.0006306023268302053, -2.869535499267501e-18, \
-0.0000261496152121679, -5.0247589966073113e-17, \
2.0251425458487276e-17, 2.2895761376976727e-18, \
-4.026672293697236e-8, 0.0001582178114170554, \
-0.00002628623036345268, 9.84527511871546e-18, 1.4541114252387646e-6, \
1.2785096453701482e-17, 8.697946586664446e-18, \
-2.2290506075812504e-17, 5.786101139990347e-12, \
-3.0239408137021064e-17, 7.043664500264831e-18, \
1.583038677761482e-17, -8.316561367482553e-19, \
3.6617078693063915e-17, -2.0716837095160676e-17, \
-5.8566842280942296e-18, -1.0661811612929761e-10, \
2.415652136013455e-7, -4.0266722943481733e-8, \
-3.0934201703304086e-18, 2.5621413924694933e-9, \
-0.0009523338813055688, 0.00015508867446747734, \
0.0001549986996417968, -4.627365333717561e-18, \
0.00015821781141701417, -0.000026149615212167908, \
-6.276984959294112e-19, 1.0873723106070184e-6, 8.406296919431261e-18, \
4.9414729836623135e-18, 1.926685453474312e-20, 2.562141386221813e-9, \
-8.74612296720473e-6, 1.4541114252344418e-6, -8.68537230387153e-19, \
-8.304944942582978e-8, -0.13970951381782767, 0.02275136494773174, \
0.022751378955960872, 7.546753001920351e-18, 0.022751364947731778, \
-0.0037597451955120663, -2.042626917371799e-17, \
0.00015508514031014817, 0.022751689048796406, -0.003703643733268825, \
-0.0037598014822431513, -8.603733279015359e-18, \
-3.290882731997536e-17, 1.3130253872267377e-17, \
0.0001550886744674983, 3.2251077099770287e-18, 0.022742687730454687, \
-0.003703643733268821, -0.003702181051036716, -9.139403415196667e-18, \
-0.0037583021569814927, 0.0006211270496211923, \
1.1783694459224805e-17, -0.000025755425821062117, \
1.78363219267145e-17, -1.3591131420387959e-19, \
4.5689441398623134e-18, -3.9311197477432953e-8, \
0.0001549986996418351, -0.000025750975332157997, \
-1.3420819505232814e-17, 1.4234489168353928e-6, 0.023087514789948336, \
-0.0037597451955120992, -0.0037598014822432, 1.8512132155668365e-18, \
-0.003758302156981503, 0.0006210711239844171, 1.0520362197291712e-17, \
-0.000025613969573398497, -0.0038156294832040957, \
0.0006211270496212058, 0.0006306023268302126, 1.280385209768992e-17, \
6.313666685240115e-18, -2.7148809176856353e-18, \
-0.00002614961521215819, 8.593199364935615e-18, \
1.8794697737893117e-17, 1.229529693722627e-18, \
1.1402778070441874e-17, 1.1968378834803846e-18, \
4.5331377099864256e-18, 2.0429561871824156e-18, \
-4.026672294923659e-8, -7.56505252692936e-18, 0.00015821781141706493, \
-0.000025750975332155747, -0.000026286230363447375, \
-3.784215283898829e-19, -5.1206067412206966e-18, \
-1.3126881815732174e-18, 1.454111425240724e-6, \
-1.502346102650766e-18, -8.646653113811892e-18, \
-3.778452221771989e-17, -3.3392425547083364e-17, \
-6.4405571479907346e-18, 4.482314369257017e-17, \
-9.863387968531737e-18, 5.786080985049943e-12, 8.206725764280642e-18, \
4.358166159036225e-18, 5.220133754982545e-18, 2.9164106352263796e-18, \
6.846038402263946e-19, 1.3198787548098155e-17, \
-6.265798152247591e-18, 6.645467016355274e-18, \
-5.086380230136524e-18, -1.712741478524172e-17, \
3.776614777148073e-18, 1.4361404428007608e-17, \
1.4150884571234853e-18, 3.433834304686889e-18, \
-1.2059150728988544e-17, -1.0661811348737018e-10, \
4.5186158844106065e-18, 2.415652135686968e-7, -3.9311197481026127e-8, \
-4.026672293413037e-8, 4.355902417523662e-18, 5.747739168569271e-18, \
-7.080453833539117e-18, 2.562141397767073e-9, 1.263728721299709e-17, \
-0.0009523338813056611, 0.0001550851403101901, \
0.00015508867446750995, -4.685415549206517e-18, \
0.00015499869964184253, -0.00002561396957340231, \
-6.448896439720753e-19, 1.0562677716617375e-6, \
0.00015821781141705772, -0.000025755425821073917, \
-0.000026149615212170216, -6.181760266475573e-18, \
3.7781803736124215e-19, -3.2361076600585498e-18, \
1.0873723106161062e-6, -1.1222546494113614e-17, \
-3.7799473494187164e-17, 1.4423425745461253e-17, \
-5.1412384263568005e-18, -9.313704736590716e-19, \
5.370142370694184e-18, -5.6023037882865104e-18, \
2.5621413909890663e-9, -2.4321760011439683e-18, \
-8.746122967204286e-6, 1.4234489168277585e-6, 1.4541114252348816e-6, \
6.4697892786637065e-18, 2.0772846698419073e-18, \
9.910028185045534e-18, -8.304944942445096e-8, \
-4.1128717891347565e-18, -0.13970951381782817, 0.02275168904879649, \
0.02275136494773194, -2.176789193910943e-17, 0.022751378955961066, \
-0.003759801482243193, -1.1425172005539057e-17, \
0.00015508867446747683, 0.02275136494773181, -0.0037036437332688345, \
-0.003759745195512081, 1.0839900095207813e-17, 1.70014967520557e-17, \
-1.0220701280869638e-18, 0.00015508514031016934, \
-1.0099380789485573e-17, 0.02275168904879664, -0.003705162391755273, \
-0.003703643733268879, 8.511777923732434e-18, -0.003759801482243243, \
0.0006213837763661125, 4.74298835390029e-18, \
-0.000025766311028458658, -1.7314470263296268e-17, \
2.4698470287375032e-18, -4.330330269543215e-18, \
-3.934046356784252e-8, 0.0001550886744675047, \
-0.000025766311028462114, 2.522695998517749e-18, \
1.4243496326041678e-6, 0.02274268773045464, -0.003703643733268839, \
-0.003703643733268875, 6.649532171205663e-18, -0.0037021810510367375, \
0.0006118057332472542, 4.6051148674096995e-18, \
-0.000025232079350480188, -0.0037583021569814797, \
0.0006118037553734741, 0.0006211270496212101, 7.911765490653643e-18, \
-7.923379526305036e-18, -4.667433699134923e-18, \
-0.00002575542582106443, 1.090910107566366e-17, \
-2.7238498357068505e-17, 5.351980579300213e-18, \
7.549441710924703e-18, -7.805617471633433e-18, 1.817658391094242e-18, \
-2.1022713757797396e-18, -3.931119748144606e-8, \
2.5729959211541927e-18, 0.00015499869964181835, \
-0.0000252274190079651, -0.00002575097533214277, \
-6.458490185464421e-18, 3.6424308303817615e-18, \
1.0457913350981329e-19, 1.4234489168294555e-6, \
-1.381583057277926e-18, 0.023087514789948478, -0.003759801482243211, \
-0.003759745195512127, 9.494887510269728e-18, -0.0037598014822431912, \
0.0006213298458967409, 1.2277888515293448e-17, \
-0.000025629524825208806, -0.0037583021569815027, \
0.0006118057332472541, 0.0006210711239844147, \
-1.3323406558151213e-17, 3.551559648290954e-18, \
-1.026549453570751e-17, -0.00002561396957340624, \
-6.334622474939389e-18, -0.0038156294832041473, \
0.0006213837763661101, 0.0006211270496212074, -9.116236353807851e-18, \
0.0006306023268302177, -0.00010421997760641669, \
-3.281222144875483e-19, 4.321794037296324e-6, 7.389225115987556e-18, \
-1.3556190232493817e-17, 5.585053557905686e-18, \
6.6555542320625904e-9, -0.000026149615212166173, \
4.344545716613291e-6, -1.0602532997205201e-17, -2.403370941397186e-7, \
-3.392645742370772e-17, 5.975375969644024e-18, \
1.2691294687407345e-17, -9.706472380393796e-13, \
1.3710573007770629e-17, -2.5510571085479898e-18, \
1.6481784518610693e-18, -8.82234138126227e-18, 4.51347232098977e-18, \
-1.3721164581429519e-17, -1.3296622466646367e-17, \
1.778896510465254e-11, -4.02667229508539e-8, 6.712232764188477e-9, \
-9.008679487046114e-18, -4.271995417374495e-10, \
0.0001582178114170563, -0.000025766311028464198, \
-0.00002575097533214958, -3.8163692342916885e-18, \
-0.000026286230363459884, 4.344545716591273e-6, \
-8.594020923511077e-18, -1.8066117078633553e-7, \
-6.361265140089023e-18, -1.2830800134123574e-17, \
-1.0518972879275109e-17, -4.2622079213894623e-10, \
1.4541114252324921e-6, -2.41761490307628e-7, -7.986652868287647e-18, \
1.3809685157659562e-8, -6.160944433374253e-17, 6.250247096270602e-19, \
-3.2915810541108436e-17, 5.786067495369779e-12, 8.47833147872536e-18, \
-1.3589043633882214e-18, 4.084316821642386e-18, \
5.076972227966967e-18, -1.4549820827577954e-17, \
1.3662690415159952e-18, 1.0516490263182747e-17, \
-9.706409170250543e-13, 5.786083403832868e-12, \
-9.706346275002698e-13, -4.6912263761597574e-18, \
1.355891846705878e-13, 1.9913760767074225e-17, \
4.8036495238202676e-18, 3.637599992006417e-18, \
-9.406748190735813e-13, -2.700117404526145e-18, 7.55627292644389e-18, \
-4.759685140318436e-18, -6.651953557437864e-18, \
1.1074476182182517e-17, -3.681078778185036e-18, \
2.2716916727512263e-18, -2.1529644414900588e-18, \
3.711298019255711e-18, 8.474833329035047e-18, 4.370699070480784e-18, \
-2.395445673284323e-18, -3.018709990028093e-17, \
1.4329603376760515e-17, 2.0492024431621336e-17, \
-9.706398428568704e-13, 6.173464176561131e-18, 7.906737505954101e-18, \
1.1538644906954704e-18, -1.177480523869667e-19, \
1.6834136234256517e-17, 1.7222424000829192e-18, 7.68010385597213e-18, \
2.1702903426179663e-13, -1.0661811064638718e-10, \
1.7788997131031878e-11, 9.176461435040022e-18, \
-1.1811357467887782e-12, 2.41565213589195e-7, -3.934046356589954e-8, \
-3.931119747841064e-8, 2.96986334752393e-19, -4.0266722941894155e-8, \
6.655554236468675e-9, -7.826038306993564e-18, \
-2.7735250866073383e-10, -1.47489878369663e-18, \
1.9309686807100024e-18, 8.947216307204774e-18, \
-8.366330603643068e-13, 2.5621413996067724e-9, \
-4.2622076171907014e-10, 5.515252499545118e-18, \
2.4921679760321484e-11, -0.0009523338813056896, \
0.00015508867446750713, 0.00015508514031015763, \
-8.607281527409356e-19, 0.00015508867446749954, \
-0.000025629524825210873, -9.286535101898151e-18, \
1.0572126795965326e-6, 0.0001549986996417974, \
-0.000025232079350480737, -0.000025613969573395082, \
-5.237070495334169e-18, 4.171288046684991e-18, \
-5.378662660265398e-19, 1.056267771655247e-6, 8.442812815204302e-18, \
0.00015821781141705314, -0.000025766311028463015, \
-0.000025755425821067, 9.502355655023767e-18, \
-0.000026149615212163368, 4.321794037325063e-6, \
2.9207842935939084e-18, -1.7922180365586938e-7, \
1.1967895871764754e-17, -2.6290726606807097e-18, \
2.7291743715942094e-18, -2.7735250909485673e-10, \
1.0873723106177266e-6, -1.8066117077922427e-7, \
-2.694632051767375e-18, 9.998581542344997e-9, -8.023321477417008e-18, \
-7.987922284239952e-18, -5.4327228395473404e-18, \
1.3558750069258062e-13, 6.143251414385691e-18, 9.363850792082274e-18, \
-1.0929486950444106e-19, -6.852457216538418e-18, \
-5.8061168973010465e-18, -7.484109144894566e-18, \
3.780263598197828e-18, -1.1811318646859087e-12, \
2.5621413956795675e-9, -4.2719952647335755e-10, \
-8.223987551229365e-18, 2.7435213800719603e-11, \
-8.746122967228411e-6, 1.4243496325994049e-6, 1.4234489168275836e-6, \
9.718723899847937e-18, 1.4541114252351978e-6, -2.4033709416373484e-7, \
-1.8011603233712693e-18, 9.998581554001937e-9, \
-8.090085315388401e-18, -9.921280776502712e-18, \
6.780946916815328e-18, 2.4921708262239826e-11, -8.304944943267321e-8, \
1.3809685126179317e-8, 5.910605435596125e-18, -7.930828972825087e-10, \
-0.13970951381782812, 0.022742687730454667, 0.02275168904879646, \
1.6102510985311328e-18, 0.022751364947731865, -0.003758302156981517, \
1.6305107241119623e-18, 0.00015499869964180548, 0.022751378955961014, \
-0.0037021810510367145, -0.003759801482243168, \
-1.8480462715050422e-17, 6.248763924904835e-17, \
-1.2590868519071298e-17, 0.00015508867446748333, \
-1.217547376281093e-18, 0.022751364947731983, -0.0037036437332688813, \
-0.003703643733268856, 9.506391239719681e-18, -0.0037597451955121, \
0.0006211270496212106, -5.038765667362606e-18, \
-0.000025750975332145928, -2.8084259263551e-18, \
1.624946584624722e-18, 6.884324757968971e-18, -3.9311197475713794e-8, \
0.0001550851403101885, -0.000025755425821071732, \
-1.012656716432648e-17, 1.4234489168342283e-6, 0.022751689048796504, \
-0.003703643733268851, -0.0037051623917552464, \
1.1529707320526254e-17, -0.0037036437332688176, \
0.0006118037553734804, -3.491786884053557e-18, \
-0.000025227419007968436, -0.0037598014822431847, \
0.0006118057332472544, 0.0006213837763661007, 2.0710625604679446e-19, \
-1.6973784975356963e-17, -5.2501705810605145e-18, \
-0.000025766311028452458, -9.722466309353889e-18, \
4.345738206082047e-17, -1.4304407527843168e-18, \
-1.0313002256962539e-17, -2.8854949248874406e-18, \
-1.1965108592553342e-17, -9.673831704251806e-18, \
-3.934046357316988e-8, 5.126566560999101e-18, 0.0001550886744675186, \
-0.000025232079350486653, -0.00002576631102846153, \
-3.98318320722189e-19, 1.3756849147850759e-17, \
-1.408153442675787e-17, 1.424349632600143e-6, 1.0482240963911243e-17, \
0.02274268773045463, -0.003702181051036714, -0.003703643733268861, \
-5.829991880744429e-18, -0.003703643733268855, 0.000611805733247254, \
-5.658408837670718e-18, -0.000025232079350484894, \
-0.0037021810510366976, 0.0006024315074068216, 0.0006118057332472522, \
-6.657917244262301e-18, -1.6125011715740974e-17, \
-5.0959599473018335e-18, -0.00002523207935048376, \
-1.6541774299978944e-17, -0.0037583021569815196, \
0.0006118057332472551, 0.0006118037553734867, 9.197857793271055e-19, \
0.0006211270496212067, -0.00010261311534407326, \
-5.867952488183091e-18, 4.254383932943637e-6, -4.870886275750871e-20, \
9.021431942411786e-18, 1.0466167978706417e-17, 6.5502825314313394e-9, \
-0.000025755425821065884, 4.27734335764705e-6, \
-7.457613032231719e-18, -2.3656816857230745e-7, \
-4.198655654540888e-17, 6.531100761441496e-18, \
1.3613903441433618e-17, -9.40679721085262e-13, 8.1501385834564e-18, \
6.679510950584241e-18, -5.842892627109437e-18, \
-4.157763347403796e-18, 5.8851875524930734e-18, \
-2.0380503544916808e-19, -9.52316394788633e-18, \
1.7337278504898245e-11, -3.931119748525543e-8, 6.550282511827725e-9, \
1.0636016926848154e-17, -4.1669323510073734e-10, \
0.00015499869964180532, -0.000025232079350480293, \
-0.000025227419007963188, 9.24873754256799e-18, \
-0.00002575097533214419, 4.254383932941196e-6, 7.517620437689893e-18, \
-1.7687783263753447e-7, 7.176672643953461e-18, 3.820214231335185e-18, \
-4.011639458377773e-18, -4.1669323805762213e-10, \
1.4234489168292554e-6, -2.3656816858132181e-7, 7.452671749909588e-19, \
1.350840760657932e-8, 0.023087514789948325, -0.003758302156981491, \
-0.003759801482243163, 3.3241818987830525e-18, -0.00375974519551207, \
0.0006210711239844107, 1.5316541882631787e-17, \
-0.000025613969573389644, -0.0037598014822432073, \
0.0006118057332472559, 0.0006213298458967347, -4.267691943347172e-18, \
-7.4996471318311085e-19, 1.5614942912711998e-18, \
-0.000025629524825201915, -9.147661797708272e-18, \
-0.003758302156981525, 0.0006118037553734814, 0.0006118057332472477, \
-1.233638513768987e-17, 0.0006210711239844084, \
-0.00010260346926310202, -1.7797640146439526e-17, \
4.25376858086615e-6, -5.820300083145553e-19, -1.0415151675237899e-18, \
5.751980240140645e-18, 6.4917769630035484e-9, \
-0.000025613969573403993, 4.253768580847165e-6, \
1.2766895155835557e-18, -2.3509007879627597e-7, \
-0.0038156294832041105, 0.0006211270496212059, 0.0006213837763661049, \
-2.349737781426548e-18, 0.0006211270496212011, \
-0.00010260346926309287, 1.301023708860329e-18, 4.230789067435551e-6, \
0.0006306023268302174, -0.00010261311534405606, \
-0.00010421997760642181, -1.0348981773088845e-17, \
1.1032794266546529e-17, -5.724734067976779e-18, \
4.3217940373340445e-6, -5.9426778164652695e-18, \
-1.1557404882770002e-17, -2.24959028184859e-18, \
-7.489631967005317e-19, 1.3604238463899073e-17, \
-1.4277618535256634e-18, 6.681221411737566e-18, 6.655554236627812e-9, \
-1.8104247406344908e-17, -0.000026149615212168318, \
4.254383932957225e-6, 4.34454571660503e-6, -1.124485539803865e-17, \
-6.780440149989757e-18, -7.875686808291553e-18, \
-2.403370941654397e-7, 6.123072612797702e-18, 3.630676469020853e-17, \
-7.676754286431258e-19, 7.452737414314133e-18, \
2.1830564358199365e-18, -7.988696553222032e-18, \
8.695171165226505e-18, -9.706435334154922e-13, \
-1.2341237067180936e-17, -3.2230856823608726e-18, \
-7.456970299005797e-18, -4.1164329505002454e-18, \
-9.748929189685588e-18, -8.19144228856926e-18, \
-1.0282556357136601e-17, -7.177903348782623e-18, \
-1.922676858797545e-17, -7.35513329667436e-18, \
-3.6995536473643904e-18, 3.071787825915417e-18, \
3.685549997860121e-18, 3.2789606212526755e-18, 7.159184051833125e-18, \
1.7788984587543784e-11, 1.0462488629047044e-17, \
-4.026672293463231e-8, 6.5502825033695895e-9, 6.712232765312125e-9, \
-1.5005265696790063e-17, 6.2220005878454244e-18, \
1.2351218139140038e-17, -4.271995297076313e-10, \
8.881681262102477e-18, 0.0001582178114170143, \
-0.000025755425821069113, -0.000025766311028457045, \
-1.0719050777218337e-17, -0.00002575097533214576, \
4.253768580870916e-6, 9.280045254307294e-18, -1.7538575990497414e-7, \
-0.000026286230363455188, 4.277343357621025e-6, \
4.3445457166024826e-6, -5.624786269151263e-18, 5.720227358053359e-18, \
1.2397860978984334e-17, -1.806611707596672e-7, \
1.8633256021173034e-17, 1.607136009676695e-18, \
-4.448362329937421e-18, -3.3974917775995564e-18, \
-9.217517106139568e-18, 8.30924687051802e-18, 1.4176385692629265e-17, \
-4.26220785407926e-10, 1.4701355071532e-17, 1.4541114252369444e-6, \
-2.3656816857692526e-7, -2.417614902927834e-7, 2.707503347606412e-18, \
1.3032175466050475e-17, 8.555657973415243e-18, 1.3809685151518076e-8, \
-7.288619277111775e-18, 4.634276089207397e-17, \
-4.9701731073053046e-17, -9.16117437062782e-18, \
-5.478880601511215e-18, -3.905034699907461e-17, \
4.007728520571675e-19, 5.786055385801256e-12, 8.72371588617536e-18, \
-2.7573780027673984e-17, 1.2770123399719487e-17, \
-4.568015458996088e-19, -4.727616097868569e-18, \
-2.0507804424815566e-18, -4.785488109343735e-19, \
-2.318817280632298e-18, 3.081645383780794e-19, \
-3.767497833094244e-17, 1.612069460247629e-17, 5.706622521569864e-18, \
1.2028016528699732e-18, 1.9624967855753423e-17, \
2.6271504901915347e-18, -9.706354990139414e-13, \
-7.130848709198628e-18, 5.7860645806939405e-12, \
-9.406767471986605e-13, -9.70640055343388e-13, 7.315245086757855e-18, \
-9.199385337662213e-18, -1.106825379830679e-18, \
1.3558461081790876e-13, -6.8441459289898485e-18, \
2.7487543773004758e-17, -4.4713191057189964e-18, \
-4.109623283327406e-18, -1.2187311081315278e-18, \
-3.807418056705901e-18, -2.9753174664896284e-18, \
-9.406757228460717e-13, 2.7284228731558237e-18, \
-1.0460755986151697e-17, -4.411168979181859e-18, \
4.174106260289698e-18, -1.2172492943436465e-17, \
1.5922565019904101e-18, -9.752934975430452e-18, \
1.2847833837235368e-18, 1.1149232487132145e-17, \
6.606467203809296e-18, 2.417918744629256e-18, 8.034384479335784e-19, \
-8.485785973465166e-18, 3.1000871976494903e-18, \
9.737738120003348e-18, -1.3114391272217984e-17, \
-5.860049851825176e-18, 4.236794135034449e-18, \
-4.178882127780715e-18, -6.895403122338129e-18, \
1.5510940694484614e-17, -2.703426053786538e-18, \
-1.806472538851519e-17, 5.625312033376651e-19, 3.9064073568549e-18, \
1.6109991659503104e-18, 2.7529918988874e-18, -9.606197922802141e-18, \
1.2921166731913708e-18, 7.591658457621391e-18, \
-2.6482849428302418e-18, -9.70634577085156e-13, \
-5.341076340113635e-19, 6.726115753275314e-18, \
-1.7611713140544813e-17, -2.0480437500341783e-18, \
-1.4065109013512772e-17, -1.5489394741545138e-18, \
9.175462750043913e-19, -1.0480438833840386e-17, \
-5.643626160360153e-18, -8.790000484952037e-18, \
-2.943085677645457e-18, -7.42706838413337e-18, \
1.0366403200867727e-17, 4.441717499724804e-20, \
1.9712891933908962e-17, 2.1703157125682677e-13, \
5.198766539815907e-18, -1.066181124800123e-10, \
1.7337268184609224e-11, 1.7788987792929313e-11, \
-1.9153144487152886e-17, 2.0278230770102652e-18, \
-1.6746649682466716e-17, -1.1811442638832744e-12, \
1.22071479423906e-17, 2.4156521358322455e-7, -3.931119747857683e-8, \
-3.9340463560432394e-8, 3.699824898348643e-18, \
-3.9311197481447784e-8, 6.49177694762585e-9, -4.4543330301349075e-18, \
-2.675991621876183e-10, -4.02667229386107e-8, 6.550282513193827e-9, \
6.655554244324262e-9, -1.2100167408741035e-17, 1.457157696197987e-19, \
-7.686930999263053e-18, -2.7735251376967075e-10, \
1.2121241933330836e-17, 5.1938914558715185e-18, \
5.558566997346998e-18, -1.1971199473420766e-17, \
6.940731038288887e-18, -8.309962150159123e-19, -8.800452134375e-19, \
-8.366391389375043e-13, 2.4795371322597813e-18, 2.562141393536667e-9, \
-4.1669322731130417e-10, -4.262207744254825e-10, \
-1.4903531872637344e-17, -1.5451187578961944e-18, \
-1.806384760949824e-17, 2.4921703536348298e-11, \
1.1710440061087762e-17, -0.0009523338813055946, 0.000154998699641784, \
0.00015508867446748515, -9.292069035462812e-18, \
0.00015508514031013787, -0.000025613969573392995, \
-1.0543348687774407e-17, 1.0562677716556774e-6, \
0.00015508867446751762, -0.00002523207935048046, \
-0.000025629524825208057, 2.606078999800958e-18, \
7.648179967005106e-18, -8.176498192536992e-18, 1.0572126795991184e-6, \
8.372371593978651e-18, 0.00015499869964180139, \
-0.00002522741900796462, -0.00002523207935048189, \
3.086236500787382e-18, -0.000025613969573392405, \
4.230789067413153e-6, -7.342689209818852e-18, -1.7538575991808224e-7, \
7.121200155422558e-18, -3.3016468675208235e-18, \
9.651526836178903e-18, -2.675991767931421e-10, 1.0562677716540651e-6, \
-1.7538575991438893e-7, 1.6347494447166824e-18, 9.691788329176824e-9, \
0.00015821781141700866, -0.000025750975332141154, \
-0.000025766311028448985, 9.11169706029436e-18, \
-0.000025755425821063797, 4.25376858084492e-6, \
-4.324742753967296e-18, -1.753857599114662e-7, -0.00002614961521216, \
4.2543839329729825e-6, 4.32179403733391e-6, -1.9962879037694088e-17, \
5.738837504312704e-19, 1.14135715701844e-17, -1.7922180365448903e-7, \
-6.963024060388906e-18, -6.043750321476434e-18, \
-3.677315672698224e-18, -4.0824462385646955e-18, \
1.1417028866852707e-17, -1.0495301570486076e-17, \
1.2584803577688897e-17, -2.77352483272933e-10, 1.725704567153213e-17, \
1.0873723106101838e-6, -1.7687783264182437e-7, \
-1.8066117078521904e-7, -1.9256858228899435e-18, \
-8.36727645846853e-18, 1.5384393406668337e-17, 9.998581561546181e-9, \
1.9240138363942818e-17, -3.8755562691555727e-17, \
7.466997452706656e-18, 1.6312944948504282e-17, \
-2.478647049046268e-19, 1.533430630845562e-18, 7.565036025194464e-18, \
1.3558038099158293e-13, 1.8760554333978817e-20, \
3.783495593750578e-18, -1.2095715206151857e-17, \
-3.752416245035603e-18, -5.317533165857995e-18, \
-1.4510717181490608e-18, 6.385287743990734e-18, \
1.1624240726809903e-19, -1.1995081042491117e-18, \
1.3078198997531988e-17, 3.0337372619142993e-18, \
-6.100629588964933e-18, 4.912537501001162e-18, \
-5.853541965910008e-18, 2.4188134682554642e-17, \
-1.1811544867112443e-12, 2.339717332600294e-18, \
2.5621413925546234e-9, -4.166932193737108e-10, \
-4.2719953846559465e-10, -1.595700903363724e-17, \
-1.1860167417407995e-18, -1.4465227651017917e-17, \
2.7435214751560957e-11, -9.400844888769341e-18, \
-8.746122967163467e-6, 1.423448916823838e-6, 1.4243496325976678e-6, \
5.917790098276246e-18, 1.4234489168217515e-6, -2.350900787966026e-7, \
-7.762404079563632e-18, 9.691788324105145e-9, 1.4541114252337237e-6, \
-2.365681685768896e-7, -2.4033709414341076e-7, \
-4.814774164875403e-18, 9.846159360465733e-18, \
1.0333635584029264e-17, 9.998581549503932e-9, \
-1.9500617428607814e-18, 1.6544411547077877e-18, \
-1.1030935974489089e-17, -1.8885621594309342e-18, \
1.4421653658691797e-17, 9.224631466151796e-20, 5.211655668724779e-19, \
2.492169900798666e-11, -8.86005413452701e-18, -8.304944943009074e-8, \
1.3508407615223151e-8, 1.3809685137471075e-8, 1.9842299220831263e-17, \
9.99563929283334e-18, -2.6119349512576885e-18, -7.93082913821038e-10, \
1.551475055866399e-17, -0.1397095138178281, 0.023087514789948412, \
0.02274268773045462, -5.724974954474811e-18, 0.0227516890487965, \
-0.003815629483204153, -2.1765638657668105e-17, \
0.00015821781141703176, 0.022751364947731834, -0.003758302156981487, \
-0.0037583021569814797, -1.2763876991424275e-18, \
1.6908070047692823e-18, 1.5860068782046698e-17, \
0.00015499869964181792, -1.2314221105853435e-17, \
0.022751378955961063, -0.0037598014822432073, -0.0037021810510367266, \
8.65087981925892e-19, -0.003759801482243209, 0.0006306023268302229, \
3.976038583595849e-19, -0.000026286230363447216, \
1.4714856491511064e-17, 1.1055651916954298e-17, \
1.3439686507648297e-18, -4.02667229402066e-8, 0.00015508867446750285, \
-0.00002614961521216996, -7.359481819675985e-18, \
1.4541114252346625e-6, 0.022751364947731816, -0.0037597451955120793, \
-0.0037036437332688354, 2.097988035712253e-18, \
-0.0037036437332688727, 0.0006211270496212165, 9.708454208572215e-18, \
-0.000025750975332154995, -0.0037597451955121096, \
0.0006210711239844159, 0.0006211270496212032, -5.245213396118063e-18, \
4.566071820140059e-18, 7.904798254309312e-18, \
-0.000025750975332147548, -9.156852553997057e-19, \
-1.237559797314682e-17, 7.436706540344994e-18, 8.841849572447585e-18, \
-2.7984567542246815e-18, 4.660202165879796e-18, \
3.810949294425601e-18, -3.931119749028653e-8, 2.562357391907725e-18, \
0.00015508514031019167, -0.000025613969573401757, \
-0.000025755425821064278, 2.6324860280398043e-18, \
-8.719430097060571e-20, 6.870464352582635e-19, 1.4234489168308137e-6, \
6.17900159741707e-18, 0.022751689048796545, -0.0037598014822431943, \
-0.003703643733268857, 3.4827298534884956e-18, \
-0.0037051623917552386, 0.0006213837763661095, 5.838369112974395e-18, \
-0.000025766311028456395, -0.0037036437332688553, \
0.0006118057332472511, 0.0006118037553734844, 1.0559041453330345e-18, \
-3.478964057508038e-19, 2.2449963455748242e-18, \
-0.00002522741900797177, 7.046698009578009e-18, \
-0.0037598014822431934, 0.0006213298458967443, 0.0006118057332472494, \
5.066471409422638e-19, 0.0006213837763661081, \
-0.00010421997760640552, 1.0248423649102125e-17, \
4.3445457166150144e-6, 1.2026454155500655e-17, \
1.7094698564737576e-18, -1.2415778406162538e-17, \
6.712232765187584e-9, -0.000025766311028460074, 4.344545716604602e-6, \
6.945882273776934e-18, -2.4176149030335714e-7, \
-1.9415567732326963e-17, 4.887408824821205e-18, \
-2.3581644463809562e-18, -9.706383909422443e-13, \
6.494219787595146e-18, -9.23507167013323e-18, -6.259762303809692e-19, \
9.710206165681084e-18, -8.608190822767227e-18, \
-2.733711083340873e-18, 1.237238362567897e-18, 1.778899865268512e-11, \
-3.9340463568080565e-8, 6.6555542312497035e-9, 9.127836323068213e-18, \
-4.26220761381509e-10, 0.00015508867446747918, \
-0.000025629524825205723, -0.000025232079350480097, \
1.2983414216859389e-17, -0.00002576631102845801, \
4.321794037298053e-6, 6.381047048396966e-18, -1.806611707804181e-7, \
-4.831451902686016e-18, 7.821634150668634e-20, \
-6.7351963653079234e-18, -4.2719954351081245e-10, \
1.4243496326054828e-6, -2.4033709414432895e-7, 7.606863559614406e-18, \
1.3809685132885463e-8, 0.02274268773045455, -0.0037583021569814537, \
-0.0037021810510366755, -2.562761532454725e-18, \
-0.0037036437332688523, 0.0006211270496212104, \
-2.6450483448760366e-18, -0.000025755425821066765, \
-0.0037036437332688085, 0.000611803755373471, 0.0006118057332472394, \
-1.336065888125589e-17, 1.5046366149589793e-18, \
4.456492750059776e-18, -0.00002523207935048074, \
-5.4671026667598616e-18, -0.0037021810510367193, \
0.0006118057332472432, 0.0006024315074068219, -7.915875327890462e-19, \
0.0006118057332472496, -0.00010261311534407815, \
-3.8283317520438725e-18, 4.277343357624925e-6, \
-9.384000448250718e-18, 2.454084885188395e-18, -6.03730806811926e-18, \
6.5502825128208245e-9, -0.000025232079350489092, \
4.254383932944675e-6, -4.658362860597845e-18, -2.3656816857586983e-7, \
-0.003758302156981466, 0.0006210711239844043, 0.0006118057332472413, \
-5.988227636500291e-18, 0.0006118037553734811, \
-0.00010260346926309672, -7.06358550555074e-18, 4.253768580842307e-6, \
0.0006211270496211983, -0.00010260346926309775, \
-0.0001026131153440719, -2.234775667810124e-17, \
-1.327823146725899e-17, 2.0438352025126184e-17, 4.254383932961986e-6, \
-6.235320772275426e-18, -3.0874276157478006e-18, \
-5.868643413918243e-18, -1.6067924126649176e-17, \
1.9037355673555152e-17, -1.1977118704240743e-17, \
-9.577751566057392e-18, 6.550282494882213e-9, \
-1.0594147031076576e-17, -0.000025755425821072627, \
4.253768580845152e-6, 4.277343357648216e-6, 2.134203835651579e-17, \
3.3215184510612294e-18, 1.5142580049116063e-17, \
-2.365681685770276e-7, 8.973808465361115e-18, 9.615908702214238e-19, \
4.92290108977508e-18, 7.097964602972494e-18, -2.678257748168094e-18, \
-6.053489707857755e-18, 1.1635611741998998e-17, \
-9.40678628549084e-13, -6.498951466970249e-18, \
-1.7356003271715816e-18, -7.376102845460704e-18, \
3.942246302105512e-18, -9.681145715182945e-18, \
-3.371439688615262e-18, -3.551195595511048e-18, \
3.2845707779506583e-18, -1.625741498757913e-17, \
-2.1444665814804536e-18, -3.459856400613887e-18, \
-5.4304448680699735e-18, -1.0285233591305138e-17, \
6.583060969822528e-18, -1.6526790796366753e-17, 1.73372652642394e-11, \
-1.3675918205163356e-17, -3.9311197479934056e-8, \
6.491776934365792e-9, 6.550282512427754e-9, -4.905931084061885e-18, \
6.109539895152276e-18, 7.646501489319343e-18, -4.166932236482505e-10, \
2.2549541289232885e-17, 0.0001549986996418105, \
-0.000025613969573398433, -0.00002523207935048282, \
-1.229723500802235e-17, -0.000025227419007974694, \
4.230789067426277e-6, 7.069832639376404e-19, -1.7538575989420014e-7, \
-0.00002575097533214924, 4.2537685808717635e-6, \
4.2543839329694225e-6, -4.677838445364924e-18, \
-8.867535316919146e-18, -9.840299630897212e-19, \
-1.768778326220093e-7, -2.581566149297433e-17, \
2.0753130862221412e-17, -7.309310310598857e-20, \
6.945463417024166e-18, 1.6069558684893969e-18, \
2.9491901076166693e-18, -1.4627185245631227e-17, \
-4.166932526228865e-10, 1.8687037824461734e-17, \
1.4234489168222079e-6, -2.3509007881830814e-7, \
-2.3656816859687239e-7, 1.7115736940965814e-17, \
9.100295856693431e-19, 7.238259092294422e-18, 1.3508407601410588e-8, \
9.047068991115526e-18, 0.0230875147899485, -0.0038156294832041326, \
-0.003758302156981514, -3.5777035114945156e-19, \
-0.0037598014822432203, 0.0006306023268302254, 5.19443933313939e-18, \
-0.00002614961521215677, -0.0037597451955120997, \
0.0006211270496212084, 0.0006210711239844137, 7.129446468184403e-18, \
-6.96578888992882e-18, -2.2613504231149662e-19, \
-0.000025613969573402448, 3.795382341975651e-18, \
-0.003759801482243243, 0.0006213837763661237, 0.000611805733247259, \
1.1002799970194881e-17, 0.0006213298458967613, \
-0.00010421997760641268, -2.0874980500704893e-19, \
4.344545716605968e-6, 9.143221718590632e-18, 7.908876295765039e-18, \
9.746276213381439e-19, 6.6555542239342745e-9, \
-0.000025629524825208454, 4.321794037312036e-6, \
-1.828007346818465e-18, -2.403370941672263e-7, -0.003758302156981516, \
0.0006211270496212033, 0.0006118037553734764, -8.466176917417017e-18, \
0.0006118057332472623, -0.00010261311534406122, \
-1.765011782884274e-18, 4.254383932969868e-6, 0.0006210711239844175, \
-0.00010260346926309859, -0.00010260346926309416, \
-1.0011797909423218e-18, -5.7297921649621415e-18, \
-9.380254504370686e-19, 4.253768580855186e-6, -2.181924357406825e-17, \
-1.2898650691671914e-18, 3.5126010553315655e-18, \
1.400550468477368e-17, -1.3597971354701351e-17, \
-9.011097844769385e-18, 2.002780000753386e-17, 6.491776932315153e-9, \
5.108987161301249e-18, -0.000025613969573394116, \
4.2307890674503814e-6, 4.253768580871414e-6, -1.3764535495316379e-17, \
1.124505482591478e-18, -1.3360613331178982e-17, \
-2.350900788000601e-7, -2.351297881027932e-17, \
-0.0038156294832041586, 0.0006306023268302242, 0.0006211270496212145, \
2.7922801830799485e-18, 0.0006213837763661149, \
-0.00010421997760641841, -9.919323340602252e-18, \
4.321794037318532e-6, 0.0006211270496212186, -0.00010261311534405043, \
-0.00010260346926309097, -1.9690258978530445e-17, \
7.734674945254641e-18, -1.0752186160697368e-17, \
4.2307890674160715e-6, -2.3480863054640208e-17, \
0.0006306023268302322, -0.00010421997760640566, \
-0.00010261311534406004, 6.22717027718916e-18, \
-0.00010421997760640303, 0.000017481570768016815, \
1.8633503782221445e-19, -7.287773339165004e-7, -9.12944862856889e-18, \
-1.993277657226673e-18, -2.4053899370692257e-17, \
-1.1260840761973197e-9, 4.321794037323465e-6, -7.287773338925337e-7, \
-1.2457229966363712e-17, 4.055693173526915e-8, 9.339149071711013e-18, \
-8.273567866162118e-18, 3.3967033942955946e-18, \
2.1703828922945134e-13, -6.760108304425175e-18, \
-5.327961467036307e-18, 1.73513759243637e-17, 5.19234648241258e-18, \
3.205781441540035e-18, -1.2932074418396721e-17, \
-1.1133653947388873e-18, -3.021691385544128e-12, \
6.6555542266299535e-9, -1.126084044784977e-9, \
-1.8002544530509658e-17, 7.213669717411057e-11, \
-0.000026149615212168372, 4.321794037324472e-6, 4.254383932946716e-6, \
7.602732969187134e-18, 4.344545716601154e-6, -7.287773338959827e-7, \
-2.034519766613622e-17, 3.046638533847834e-8, 9.52815610878876e-18, \
-1.1885036813148588e-17, -1.8617279504000064e-17, \
7.213665267022322e-11, -2.403370941452261e-7, 4.055693175066964e-8, \
-2.6338442380521805e-17, -2.330812980104046e-9, \
3.105229320730224e-19, 9.164360996378606e-19, 4.168861557402816e-18, \
-1.0661811216735022e-10, 3.0284410234687387e-18, \
-1.0082068033259564e-17, -1.2517170494843876e-18, \
-6.318476063477045e-18, 2.854815459412996e-19, 6.174507276156839e-18, \
7.262617311442048e-18, 1.7788967110129543e-11, \
-9.706383334749497e-13, 2.1703718209494037e-13, \
-4.799210037771498e-18, -1.1811307491125124e-12, \
-4.229610234709821e-18, -1.0945368008477898e-17, \
-3.9807776594855444e-18, 1.7337274878610956e-11, \
-5.348927544249686e-18, -7.949212984527793e-18, \
-9.383305334437842e-18, -1.7692883321551168e-17, \
-1.6329496061092949e-18, -5.30989142550553e-19, \
1.1168739435531948e-17, 1.3848582761191558e-17, \
-9.87261782946521e-18, -1.105021053080378e-17, 2.029137455525231e-17, \
6.273896336615221e-18, 1.1433271348379707e-17, \
-4.995236067344035e-18, -2.454087500702783e-18, \
1.7788980758111513e-11, 4.239362588657359e-18, \
-1.8140267341682773e-17, 7.039677792712418e-18, \
4.642500099081915e-18, 1.7171218078541573e-18, \
4.6140477636412035e-18, 1.7989802564440755e-17, \
-3.021704618398193e-12, 1.778899040222448e-11, \
-3.0216975410879162e-12, 9.653443613108267e-18, \
3.988407546990041e-13, -4.026672293817913e-8, 6.6555542296804145e-9, \
6.550282529880703e-9, -1.0362802848026052e-17, 6.712232738497756e-9, \
-1.1260840859085088e-9, -1.0275385580149233e-18, \
4.717936237846388e-11, -2.3485011415928825e-19, \
-1.9199740544964912e-17, -7.048338050284938e-18, \
2.687455182439032e-13, -4.271995131137226e-10, 7.213668311036396e-11, \
1.2090882897529085e-17, -4.256325591592525e-12, \
0.00015821781141702108, -0.000026149615212168867, \
-0.0000257554258210622, -6.694069523304755e-18, \
-0.000025766311028459546, 4.321794037306374e-6, \
1.5729136775210063e-17, -1.792218036735422e-7, \
-0.000025750975332142465, 4.254383932945476e-6, 4.253768580858104e-6, \
-6.690468087538027e-18, -9.952198387210272e-18, \
-5.248934224832657e-19, -1.753857599206589e-7, 7.338927321951744e-18, \
-0.00002628623036344781, 4.3445457165949906e-6, 4.277343357641371e-6, \
-1.4948911591944615e-17, 4.344545716607665e-6, -7.287773339326635e-7, \
1.703531784867515e-17, 3.038318627841634e-8, -1.0397962639861394e-18, \
2.2460879491050457e-17, -2.161358699007845e-17, \
4.7179367035868854e-11, -1.8066117075826e-7, 3.046638534667689e-8, \
-6.105509548622303e-18, -1.696296783245452e-9, \
-8.742401148741622e-18, 1.886710854719938e-19, \
-4.9685043081195144e-18, -1.181134987145047e-12, \
4.442393096263651e-18, -2.6292916431700277e-18, \
-1.5539117196711718e-17, -1.8970858569659734e-17, \
1.8456195763665348e-18, 1.0568563368139175e-17, \
-4.115131883748277e-18, 3.9882998245148273e-13, \
-4.262207664037989e-10, 7.213665963052674e-11, \
-2.6291279638809467e-17, -4.6765761538272674e-12, \
1.4541114252368633e-6, -2.403370941570785e-7, -2.3656816859951253e-7, \
-1.0644050359867685e-17, -2.4176149030642007e-7, \
4.055693176464172e-8, 1.5924112330435042e-17, -1.6962967554756623e-9, \
1.236552654599197e-17, -6.844915021114425e-18, \
-1.6723102954633944e-17, -4.256302935426017e-12, \
1.3809685138035496e-8, -2.3308129864321244e-9, \
-1.811153800854488e-17, 1.3468706919850027e-10, \
-2.315507691748113e-17, -2.521128582123437e-17, \
4.2272512410131454e-17, 2.415652136188968e-7, \
-2.7383993267449932e-17, 1.944887981007549e-17, \
-2.6012542818188777e-18, -6.612122377229593e-18, \
1.558277826754407e-17, -1.3973908335487637e-18, \
-4.137658316026377e-18, -4.02667229466052e-8, 5.786076644514943e-12, \
-1.0661811416016189e-10, 1.2397642362756178e-18, 2.56214139463636e-9, \
-2.9979604255976056e-17, 1.7678648136434774e-18, \
-8.564248886729287e-18, -3.931119748809632e-8, 1.325292854557179e-17, \
-1.405406797644759e-18, -2.8803607486501122e-18, \
3.667523710554227e-18, -1.9794016034571245e-18, \
-2.8788086355368054e-18, -6.658655071551575e-18, \
-4.303067795463453e-18, -5.135747480722251e-18, \
-1.2538275445182829e-18, -1.1106685617718932e-18, \
3.837838154920506e-18, -2.3376183162582906e-17, \
-3.7918006660594034e-19, -6.2545468097696905e-18, \
-3.9340463569159705e-8, -4.2430363971608655e-19, \
3.7049661693096134e-18, 5.194202939339704e-18, \
2.4489593859054037e-18, 6.5380199874312514e-18, \
5.361297545503657e-19, 7.89018937779755e-19, 6.655554235859784e-9, \
-9.706438824399387e-13, 1.7788967084623173e-11, \
-6.952792793262205e-18, -4.262207795382474e-10, \
5.786076055180616e-12, -9.706424527659645e-13, \
-9.406801823604563e-13, 3.2358346676412783e-19, \
-9.706396361236654e-13, 2.1702638619807984e-13, \
-4.6149022646774005e-18, -8.36665374788971e-13, \
5.756316382837699e-18, 8.186222350812455e-18, 1.0463355098553154e-17, \
-2.7735250816200793e-10, 1.355838289336574e-13, \
-1.1811448773135389e-12, 1.1565267995248476e-17, \
2.492167413412736e-11, -2.8747736658776534e-18, \
-7.436714840408524e-19, -1.1186727667516928e-17, \
-3.931119748637218e-8, 1.490969419201933e-17, 7.289064810485254e-18, \
2.4554588024260805e-18, 2.16908003306285e-18, 6.514482161429798e-19, \
1.485523828852416e-17, -5.63947246055545e-18, 6.5502825180838935e-9, \
-9.406760441633655e-13, 1.733727263894895e-11, 6.389967585474744e-18, \
-4.166932362590988e-10, 5.531187166316536e-18, \
-3.885233756675292e-18, -9.357711975576647e-18, 6.491776937076901e-9, \
-1.3935944029278951e-19, 1.6856476908275238e-17, \
-7.457345708473105e-18, 1.3273025183697798e-17, \
-7.696805652119494e-18, -1.6112574520812504e-17, \
-1.994975675915607e-17, -8.068873726171623e-18, \
4.634905332335594e-18, 1.612798801172005e-17, -3.68742545475602e-18, \
-1.083265255235417e-17, -1.649859782601713e-17, \
6.721825182510325e-18, 6.1777624340874676e-18, \
1.2043716799821336e-17, 5.929511912253399e-18, \
1.6589430710336772e-17, 8.87829942463043e-18, 1.614284559709477e-17, \
6.876164285332907e-18, 2.5528333060674988e-18, 1.719179724560471e-17, \
1.5557689363974027e-17, -1.5938488094001498e-18, \
6.053225693584359e-19, -1.460862197348726e-17, 1.753878520942636e-17, \
4.037316461439945e-18, 6.455249041208166e-18, 6.874098261204495e-18, \
-2.675991876241862e-10, 1.2891107589098874e-17, \
1.8326581788810872e-17, 1.6466969939144182e-17, \
9.288818498247244e-18, -1.984459654846148e-18, \
-7.631879755302377e-18, 3.12893145843353e-18, \
-1.1332239720318268e-19, -3.4446299330440615e-18, \
-1.40746579807569e-17, 1.5943570796450925e-17, \
1.2610909012346855e-17, -2.939531413432355e-17, \
6.2932330825331376e-18, -1.1482934457226238e-18, \
-4.026672294874268e-8, 9.810733860548226e-18, 5.899359188407592e-18, \
4.2375981221150155e-18, 7.458692252145215e-18, \
2.0607889790807926e-18, 1.6076214185887152e-18, \
-1.3824292914525178e-17, 6.71223275226791e-9, -9.706381983487613e-13, \
1.7788980864304624e-11, -8.011353326360373e-18, \
-4.2719951373627295e-10, 9.763730443280651e-18, \
6.838931689762858e-18, -3.988977773409221e-18, 6.550282510783265e-9, \
-1.2927805278225934e-17, -2.1259783918293442e-17, \
-1.2188439841300204e-17, -5.761877236153705e-18, \
-1.4765916376607944e-18, -4.235541426392281e-18, \
-1.3385530919048868e-17, 1.9300662563877754e-17, \
1.0074508231618628e-17, 9.02232188154905e-18, -2.22360832833675e-18, \
3.1680173090627225e-18, 7.090333236788402e-18, \
-3.3463842817633836e-18, 4.300876591888424e-18, 6.655554225418878e-9, \
-4.030641438189497e-18, 1.3542478869512721e-17, \
1.5781347459038402e-17, -7.228478401877347e-18, \
-1.834902708331589e-19, 1.9659575473290887e-18, \
2.1783811571254394e-17, -1.1260840678132627e-9, \
2.1702476914665587e-13, -3.0216786221413524e-12, \
-1.7889103186379943e-17, 7.213668556633243e-11, \
-1.0661811053330482e-10, 1.7788972247036303e-11, \
1.7337266466065233e-11, -3.0317158618965493e-18, \
1.77889940687402e-11, -3.0216994276849976e-12, \
-1.2389005599670742e-17, 2.687363870038474e-13, \
7.811489451633323e-18, -9.485834222724068e-18, 2.632490812245458e-18, \
4.717939824152935e-11, -1.1811352982561842e-12, \
3.9887047646593456e-13, 1.5951681336737366e-17, \
-4.256324116235641e-12, 2.4156521357101065e-7, \
-4.0266722939326944e-8, -3.9311197476792934e-8, \
-4.7155879987336836e-18, -3.934046356947513e-8, 6.655554226174443e-9, \
-9.899477852673743e-18, -2.7735249970370613e-10, \
-3.931119747580024e-8, 6.550282503538376e-9, 6.491776952400114e-9, \
-3.4965722568860805e-18, 9.700224361730064e-18, \
-2.6884008789411094e-18, -2.675991832890187e-10, \
-2.2141483150822532e-17, -4.026672293942958e-8, 6.712232760507033e-9, \
6.55028252183054e-9, -2.2019802053814007e-17, 6.655554215340869e-9, \
-1.1260840300605642e-9, -9.359847480029009e-18, \
4.717940921716035e-11, -7.310115322276012e-18, \
-2.064016364436831e-17, -6.037104037323299e-18, \
1.4769440181878946e-13, -2.773525076322862e-10, \
4.717940574494394e-11, 2.1211101877623322e-17, -2.65080224182795e-12, \
8.29530722212011e-18, 9.9087533237592e-18, 4.0022088048442734e-18, \
-2.7735248793723407e-10, 4.0042475352982293e-19, \
2.4722651268844742e-17, -8.798443289950087e-18, 4.93372477764932e-18, \
-1.5571311305060026e-18, 1.2082250661505154e-17, \
2.146079022142159e-17, 4.717938661697175e-11, -8.366365484331307e-13, \
2.687622095177838e-13, 8.972394243756635e-18, -3.042900100236982e-12, \
2.56214139742469e-9, -4.271995314925498e-10, -4.16693225118651e-10, \
1.1890447926312043e-17, -4.2622076272839133e-10, \
7.21367001171727e-11, 2.0169752839084577e-17, -3.042873831568823e-12, \
-3.54992369304654e-18, 1.7075749267712454e-17, \
-2.2902341276855485e-17, -2.6508383019885053e-12, \
2.4921684509087687e-11, -4.25628753592627e-12, \
-2.417609904242883e-18, 4.957391638168065e-13, \
-0.0009523338813056332, 0.00015821781141703704, \
0.00015499869964181242, -2.3280083507281996e-18, \
0.00015508867446750678, -0.000026149615212163073, \
4.470486702487945e-19, 1.0873723106205218e-6, 0.00015508514031013746, \
-0.000025755425821061673, -0.000025613969573392554, \
8.824616296063876e-19, 7.545874218247845e-19, 2.652594806079644e-19, \
1.0562677716582346e-6, 4.6506989033230745e-18, \
0.00015508867446748257, -0.000025766311028456964, \
-0.000025232079350481665, -3.755752783956587e-18, \
-0.000025629524825206302, 4.321794037323826e-6, \
-7.086510065732718e-18, -1.806611707591479e-7, \
-1.3133406869818542e-18, -5.801403131861805e-18, \
-2.3548932014161387e-18, -2.773525183736754e-10, \
1.057212679595676e-6, -1.7922180365823578e-7, \
-2.6637503698874154e-18, 9.998581532240075e-9, \
0.00015499869964179442, -0.000025750975332145396, \
-0.00002522741900796621, -8.780380099388743e-18, \
-0.000025232079350479904, 4.254383932966428e-6, \
6.650368266645209e-18, -1.768778326067552e-7, \
-0.00002561396957339097, 4.253768580852638e-6, 4.230789067436741e-6, \
1.536590883474222e-17, 3.914466740791286e-18, 1.4717519444102266e-17, \
-1.7538575990329246e-7, -1.0216109434901349e-17, \
-2.4141417556516435e-18, -5.357324944787076e-18, \
5.111329514774955e-18, -1.3163238443881318e-17, \
1.4451172735033374e-18, 1.9466283346873714e-17, \
-2.6759917864298524e-10, -1.5092732365787803e-17, \
1.0562677716545719e-6, -1.7538575992535563e-7, \
-1.7538575990711551e-7, -1.604842947705051e-17, \
-4.024071125936928e-18, -4.238793581191777e-18, 9.69178831480168e-9, \
-1.449274292643623e-17, 0.0001582178114170313, \
-0.000026286230363446785, -0.000025750975332149723, \
-3.110689280577079e-18, -0.00002576631102845672, \
4.344545716614381e-6, -8.569504492808045e-18, -1.8066117075741425e-7, \
-0.000025755425821059806, 4.277343357647193e-6, 4.253768580853169e-6, \
1.8927770235681367e-17, -9.684724575910235e-18, \
-1.622410496129016e-17, -1.7538575990024933e-7, \
-1.3779419832229734e-18, -0.00002614961521216336, \
4.344545716595887e-6, 4.254383932946583e-6, 2.8468621067309884e-18, \
4.321794037322589e-6, -7.28777333915814e-7, 1.5892523045781368e-18, \
3.046638531847265e-8, -9.816553605488529e-18, 4.831629909124351e-18, \
-1.9264254831319908e-17, 4.717937826653403e-11, \
-1.7922180368263162e-7, 3.0383186280277764e-8, \
-1.0602205980807811e-17, -1.6962967857572471e-9, \
3.090275592348204e-18, 7.491530666010972e-18, 7.224316957665441e-18, \
-8.366513961365722e-13, -1.3225813909700186e-18, \
1.428140644695961e-17, 3.911592018782526e-18, 6.571605597337155e-18, \
-6.6811790934873764e-18, -1.1600319336232051e-17, \
-1.6069317129790678e-17, 2.6874266867969354e-13, \
-2.7735248374496496e-10, 4.7179383029655625e-11, \
4.2514470212391875e-18, -3.042866601110422e-12, \
1.0873723106103574e-6, -1.8066117079168942e-7, -1.768778326206584e-7, \
-2.878103951709927e-18, -1.8066117077795274e-7, 3.046638534141289e-8, \
5.24255045742636e-18, -1.2772231162640492e-9, \
-6.9226601378757844e-18, -2.2663452703251077e-17, \
-1.0798603438338986e-17, -3.0428917335977334e-12, \
9.998581555906909e-9, -1.696296770105932e-9, -2.3003471652015312e-17, \
9.78179370278072e-11, 7.392287657247485e-18, -7.151428822088061e-19, \
2.616596540824374e-18, 2.5621413942763747e-9, \
-2.8854049560684374e-18, -1.7031238834845837e-18, \
3.3277251256163075e-18, -2.0270589778963325e-18, \
-1.0114798359054457e-17, -9.177798864651017e-18, \
-6.8120226337937996e-18, -4.271995346497044e-10, \
1.3558326252726331e-13, -1.1811567422085176e-12, \
-8.805696032727577e-18, 2.743522517646496e-11, \
1.0852434219201627e-17, -8.233687985135677e-18, \
1.656864945817733e-18, -4.1669324699485324e-10, \
6.348197507761523e-18, -1.7821237617035615e-18, \
-7.468224211672202e-18, 1.906049503219462e-17, \
-9.280852974815427e-19, 2.894887725349002e-18, \
-1.1030767002066511e-17, -1.3236446284279904e-17, \
-2.059828688529193e-18, -8.350151197197588e-18, \
7.757109549576556e-18, 3.648450247686616e-18, 8.510861597393158e-19, \
8.004837834146282e-18, -8.863402791501153e-18, \
-4.2622076475069867e-10, 1.1945337848486443e-17, \
6.3362036217757716e-18, 1.8550526469288742e-17, \
-1.432804192804546e-17, -1.7422545212371246e-18, \
-3.763166800144494e-18, 8.148828333489846e-18, 7.213666404430676e-11, \
-1.1811463250180122e-12, 3.9883191474299683e-13, \
-1.3569444291000787e-17, -4.6765796242020785e-12, \
2.5621413955863872e-9, -4.2622079211286176e-10, \
-4.166932147376344e-10, 2.1008507954715165e-17, \
-4.2719953557805514e-10, 7.21366474094651e-11, \
-2.359396736720969e-17, -3.042889729773261e-12, \
3.463053940387843e-18, -2.2172637437267532e-18, \
1.7266674587603603e-18, -3.042892144625116e-12, \
2.7435247950677964e-11, -4.6765731457655085e-12, \
8.262258212267128e-18, 5.522121406060068e-13, -8.746122967179525e-6, \
1.4541114252282161e-6, 1.4234489168174459e-6, -9.43013142103396e-18, \
1.424349632598288e-6, -2.4033709414244843e-7, 6.938056865309387e-18, \
9.998581539952911e-9, 1.4234489168146718e-6, -2.365681686008797e-7, \
-2.350900788225508e-7, 7.502433512643916e-18, 9.565376355147136e-18, \
1.4728984014900385e-17, 9.691788302386536e-9, \
-2.0389764572068883e-17, 1.4541114252291756e-6, \
-2.417614903165691e-7, -2.3656816859983647e-7, \
-2.4318852825375427e-17, -2.4033709414214207e-7, \
4.055693174087522e-8, -1.1137365868141941e-17, \
-1.6962967956618457e-9, -7.610055412106027e-18, \
-2.616384561383316e-18, -6.132421324143741e-18, \
-2.6508125894913343e-12, 9.998581541030181e-9, \
-1.6962967729560303e-9, -1.4742443458114415e-17, \
9.480600087681147e-11, -4.500129044458371e-18, 1.171187445364091e-17, \
-3.738960158517481e-18, 2.4921692771025047e-11, \
-3.477739905528892e-18, -1.7840808854514933e-17, \
-1.6180270105676956e-17, -3.335901898198935e-18, \
-3.54638191646388e-18, 1.3598833070907135e-17, \
-1.3051750507212209e-17, -4.256307563257468e-12, \
2.4921706468144983e-11, -4.256274570358069e-12, \
-2.354140187919319e-17, 5.521696613978807e-13, -8.304944943394829e-8, \
1.3809685123288214e-8, 1.3508407583397312e-8, -4.858603071130795e-18, \
1.3809685133625317e-8, -2.330812972745708e-9, \
-1.0132263701153227e-17, 9.781789630146408e-11, \
1.0087672735143481e-17, 1.7894671735861256e-17, \
1.3766991537039604e-17, 4.957804599870184e-13, \
-7.930829164356964e-10, 1.3468706302608803e-10, \
-2.3856129780027213e-17, -7.845357195679485e-12, \
-0.13970951381782803, 8.077200549849624e-18, 0.023087514789948433, \
-0.0009523338813056182, 0.022742687730454653, -3.310176845977513e-17, \
-2.994969150888311e-17, 1.0949789901028303e-17, 0.022751689048796587, \
-2.1901284893676753e-17, -0.0038156294832041595, \
0.00015821781141706938, 1.0765083350563856e-17, \
2.4156521357516683e-7, 0.00015821781141705965, -8.746122967203678e-6, \
0.02275136494773183, -1.9790867204201613e-17, -0.003758302156981505, \
0.00015499869964182751, -0.0037583021569815014, \
1.0497207828498315e-17, 1.8945752094491976e-17, \
-2.9751736983133654e-18, -4.31566615566313e-17, \
1.563485486928939e-17, 6.9201237118996486e-18, \
-2.3583826142155973e-19, 0.00015499869964183624, \
-1.8490300946217073e-18, -1.420533671579629e-17, \
-9.805299575762586e-18, 0.02275137895596092, -1.818386241792469e-17, \
-0.003759801482243205, 0.00015508867446751065, \
-0.0037021810510367162, 1.5749452695639313e-17, \
1.298565772421298e-18, -3.567108293314114e-18, -0.003759801482243204, \
1.2799993011705596e-17, 0.0006306023268302289, \
-0.00002614961521217193, -3.895707587925612e-18, \
-4.026672293784183e-8, -0.000026286230363452, 1.4541114252342093e-6, \
3.6810573740617204e-17, 5.7860766662520864e-12, \
-5.748842991384826e-18, 7.859141336110408e-18, \
-2.2710269523078635e-17, -1.0661811287355572e-10, \
-4.026672295053708e-8, 2.562141393098997e-9, 0.0001550886744674985, \
-4.554323612697387e-18, -0.000026149615212162487, \
1.0873723106145312e-6, 1.6660590549493532e-17, 2.5621413952583634e-9, \
1.4541114252335725e-6, -8.30494494355266e-8, 0.022751364947731903, \
-2.621445925625607e-17, -0.003759745195512088, \
0.00015508514031016904, -0.0037036437332688583, \
-5.507766378524447e-18, 5.8848399447609035e-18, \
4.183972091786047e-18, -0.0037036437332688796, 8.37791169668134e-18, \
0.0006211270496212169, -0.00002575542582107482, 8.19253273814691e-19, \
-3.931119748212347e-8, -0.000025750975332158135, \
1.4234489168238175e-6, -0.0037597451955120775, \
1.0517278549893053e-17, 0.0006210711239844084, \
-0.00002561396957339868, 0.0006211270496212087, \
-1.0699654693885118e-17, 8.362316805884724e-18, \
-7.311862356369541e-18, 7.95133596310688e-18, -7.717675920820494e-18, \
1.2704362624363491e-17, 1.0595655470303778e-18, \
-0.000025750975332149557, -4.798809492105881e-18, \
3.999632115685469e-18, -3.468365044652517e-18, \
4.6492500316114953e-17, 8.859459482495503e-19, \
-8.162868551842893e-18, 5.20238252957991e-19, 3.519770386982715e-18, \
-2.1252135946565535e-19, -8.049550123780732e-18, \
-6.524684818135916e-18, -1.09475465524082e-17, \
-1.6981970021591838e-18, -4.363522570771431e-19, \
9.759127226183747e-18, -3.9311197487971294e-8, \
1.0572527469233019e-17, 8.03231750767753e-18, 7.743438726045291e-19, \
0.00015508514031013307, -2.991315330061019e-18, \
-0.00002561396957338845, 1.056267771655215e-6, \
-0.000025755425821059352, 1.2547880166357297e-17, \
1.5583484158887007e-17, -4.704688932695556e-18, \
-3.762657074173585e-18, 1.6800059960472575e-18, \
-2.455980176403107e-18, 1.3681343577472787e-17, 1.423448916824415e-6, \
4.160244527549153e-18, 6.679000892730207e-18, 4.63091793977166e-18, \
0.022751689048796493, -2.5940818180048566e-17, \
-0.0037598014822431847, 0.00015508867446751266, \
-0.003703643733268826, 8.374958232366778e-18, 7.717118955861103e-18, \
5.741785842658353e-18, -0.0037051623917552503, 1.061839432017485e-17, \
0.0006213837763661161, -0.000025766311028464072, \
7.59307316491629e-18, -3.934046356362881e-8, -0.00002576631102846044, \
1.4243496326012821e-6, -0.0037036437332688384, 6.564186344934644e-18, \
0.0006118057332472515, -0.000025232079350486233, \
0.0006118037553734687, 6.089039150233294e-19, \
-2.8431739771636204e-18, -2.67523460258612e-18, 6.88246007742932e-18, \
-1.3806615302967655e-18, -1.388004255405642e-17, \
-2.110930060410716e-18, -0.000025227419007975324, \
-6.2048309556901635e-18, 4.205296650880687e-18, \
5.168954203933711e-18, -0.003759801482243186, 1.0317823275543631e-17, \
0.0006213298458967529, -0.00002562952482521401, \
0.0006118057332472489, -6.9343315123352055e-18, \
1.0754730568682501e-17, 3.691216944164015e-18, 0.000621383776366109, \
-1.7751015268973993e-18, -0.00010421997760641391, \
4.321794037331828e-6, -8.400065709677658e-18, 6.655554238559468e-9, \
4.3445457165993375e-6, -2.4033709414798707e-7, \
-1.3267412998546196e-18, -9.706395002073536e-13, \
1.1734461192355087e-17, 9.684874357326469e-20, 2.336556658433763e-18, \
1.7788989906241807e-11, 6.712232742715598e-9, \
-4.2719953695599127e-10, -0.00002576631102845868, \
-7.488548944493855e-18, 4.344545716618373e-6, -1.8066117077098047e-7, \
-7.123025194446948e-18, -4.2622077467576737e-10, \
-2.417614903069157e-7, 1.3809685136338094e-8, \
-1.5744921616809566e-17, 5.786062941916963e-12, \
8.723028583637845e-18, 3.743307371555028e-18, -6.345036164017946e-18, \
-9.706372595419113e-13, -9.706476865604563e-13, \
1.3558327643292563e-13, -1.3316784220472593e-18, \
-9.40677644883716e-13, -3.736480419874855e-18, \
-4.864850601058402e-18, -4.2066157512546494e-18, \
-2.74081389802623e-19, 2.2375981640005215e-18, \
-8.280860685538543e-18, -8.819308348584681e-19, \
-9.706334157471813e-13, -4.3472416063537647e-20, \
-5.310331603113389e-19, 2.1622653749474277e-18, \
2.1703566628755925e-13, 1.7788964914397412e-11, \
-1.1811280471515445e-12, -3.9340463558340216e-8, \
7.37930929421265e-18, 6.655554242109915e-9, -2.77352514950595e-10, \
7.473011293738987e-18, -8.366479127723188e-13, \
-4.2622078015501707e-10, 2.4921707199116038e-11, \
0.00015508867446749214, 4.018456548878751e-18, \
-0.000025629524825207275, 1.0572126795938867e-6, \
-0.000025232079350486802, -2.28870765735576e-18, \
6.572499426521114e-18, -5.874509343725198e-18, \
-0.000025766311028463405, -8.198707598445438e-19, \
4.321794037327017e-6, -1.7922180368613863e-7, -6.822638356161978e-18, \
-2.773525086692007e-10, -1.806611707731262e-7, 9.998581552033717e-9, \
1.1947915020972067e-17, 1.355836425464188e-13, \
2.8225780916401874e-18, -8.068176664408496e-18, \
9.334880182781682e-18, -1.181154740776852e-12, \
-4.271995450985735e-10, 2.7435216540207452e-11, 1.424349632594421e-6, \
5.057462079066533e-18, -2.403370941618319e-7, 9.998581543035218e-9, \
9.559840347935308e-18, 2.492170050891666e-11, 1.3809685136883643e-8, \
-7.930829013106998e-10, 0.02274268773045468, -2.93604139716995e-17, \
-0.003758302156981535, 0.00015499869964180252, \
-0.0037021810510367115, 1.2767996419117878e-17, \
2.6999235910762362e-18, 2.493544980755264e-18, -0.003703643733268851, \
1.2582304892768466e-17, 0.0006211270496212132, \
-0.000025750975332153673, 9.503998240371483e-19, \
-3.9311197475218935e-8, -0.000025755425821073453, \
1.4234489168241054e-6, -0.0037036437332688627, 9.967280866033386e-18, \
0.0006118037553734867, -0.000025227419007965142, \
0.0006118057332472525, 6.7868628076765335e-19, -3.83118011440834e-18, \
-1.2859825471873623e-17, 2.9075314571233184e-18, \
-6.0193607852390576e-18, -5.002943361899599e-18, \
-8.38247497446433e-18, -0.00002523207935048236, \
8.900307790741926e-18, -1.0523107698655408e-17, \
-9.784757822986295e-18, -0.003702181051036708, 6.353248008074854e-18, \
0.0006118057332472573, -0.000025232079350481757, \
0.000602431507406825, 5.1545079908988814e-18, 2.4146187911502858e-18, \
-7.64471256983829e-18, 0.0006118057332472473, 2.485616592821231e-18, \
-0.00010261311534406389, 4.2543839329558505e-6, \
1.006570830015515e-17, 6.550282530725067e-9, 4.277343357626872e-6, \
-2.3656816860046698e-7, 3.708138948147291e-18, \
-9.406803599850236e-13, 7.241585613255782e-18, \
-1.155782809411381e-17, -8.429129944787009e-18, \
1.7337264942492297e-11, 6.550282533315006e-9, -4.166932206086757e-10, \
-0.000025232079350484715, -8.351716480019522e-18, \
4.2543839329461435e-6, -1.7687783261367868e-7, 5.021432217733508e-19, \
-4.1669322469713416e-10, -2.3656816860713678e-7, \
1.3508407599484101e-8, -0.0037583021569815214, \
6.6084096143274395e-18, 0.0006210711239844111, \
-0.000025613969573397267, 0.0006118057332472481, \
5.9229888101806235e-18, 5.805082383452403e-19, \
1.5134802902160385e-18, 0.0006118037553734823, \
-1.6141010846787355e-17, -0.00010260346926311396, \
4.253768580851111e-6, -1.099643976204418e-18, 6.491776938526893e-9, \
4.253768580844875e-6, -2.3509007880409477e-7, 0.0006211270496212059, \
-1.0374131951535033e-18, -0.00010260346926311343, \
4.230789067420595e-6, -0.00010261311534406954, 5.275302175985098e-18, \
1.5959323039362527e-17, 2.1006502596735612e-17, \
-1.0977684972042761e-17, -3.778406234458161e-18, \
-1.3682630707179068e-18, 5.694253821664209e-18, 4.254383932964559e-6, \
-7.666711812732448e-20, -1.3648010139296431e-17, \
-1.465477628174288e-17, -3.983411929322946e-18, \
3.4439449448109943e-18, -9.373312787718916e-18, \
7.882271559019647e-19, 2.4065464945490854e-18, \
1.4588505515189176e-17, 3.649486712658223e-18, \
-1.5005573698151354e-17, 1.567747584062034e-18, \
6.7136203374539365e-18, -2.0811706784815123e-17, \
-3.1380934367531023e-18, 6.550282498458847e-9, \
-1.3790909498833144e-17, 1.633920293514332e-17, \
1.9365632621278953e-17, -0.000025755425821070187, \
1.3864298252395536e-17, 4.253768580873472e-6, -1.7538575990832508e-7, \
4.277343357621618e-6, 2.4413258104702887e-17, 1.1214499200543e-17, \
-4.1743951319848335e-18, -1.0978659817164475e-18, \
9.869708955755052e-18, 1.1105119232909995e-17, \
1.7079228093438605e-17, -2.3656816859837317e-7, \
1.2687388558817094e-18, -1.4983148756668527e-17, \
-1.816409150854174e-17, -2.329145603075999e-17, \
1.0233591362610519e-17, 1.45795787788747e-17, 8.444099589890855e-18, \
-7.62810099738499e-19, 8.395742758721845e-19, -1.756898042010746e-18, \
1.0400379971819934e-18, 3.1297124283729556e-18, \
-2.6917532685531775e-18, 7.310050143609803e-18, \
9.164291150584627e-18, -9.406767808602585e-13, \
-1.7407086703669205e-18, -2.015308638320707e-18, \
1.1594855599509017e-17, 2.0796756784182905e-18, \
-1.3523233005716863e-18, 1.288376675712035e-17, \
4.356063878508083e-18, 3.746521314556453e-18, 7.536632197185912e-18, \
-1.076659338555332e-17, 3.319259346330786e-18, 6.051448656308419e-19, \
-8.854701208684311e-18, 1.3511377631314532e-17, 6.40352285490909e-18, \
-8.836462209013633e-19, -1.086740819645996e-17, \
1.295155297829024e-17, -1.169357878716967e-17, \
1.3335031864258362e-17, 2.758475357053525e-18, 4.530706474222603e-18, \
7.247426713180744e-18, -8.437780239260445e-19, \
1.6197422771839852e-17, 1.0735652233941184e-17, \
9.920486509862179e-18, 2.852040611890146e-18, 1.1725265046940048e-17, \
5.955336165360926e-18, -1.474316946718942e-17, \
1.7337271805568474e-11, 1.4489330137394218e-17, \
-1.1221314866741414e-17, 4.088794800285409e-18, \
-3.9311197481864325e-8, 4.57920873159906e-19, 6.491776944673809e-9, \
-2.6759918974574515e-10, 6.550282524339523e-9, \
-2.0767073472547688e-17, -2.1645508698072796e-17, \
-1.0302694900796562e-17, 5.556544397921032e-18, \
5.970133731250886e-18, 1.8428759891928635e-17, \
1.0926756273053865e-17, -4.166932212398395e-10, \
7.258098688074897e-18, -1.5186668940001274e-17, \
-5.099220243769753e-18, 0.00015499869964180794, \
-3.493283039134873e-18, -0.000025613969573399734, \
1.0562677716562098e-6, -0.000025232079350481137, \
8.76297754288246e-18, -5.256739369971326e-18, 7.321709471343356e-18, \
-0.000025227419007964133, 6.5883075922852335e-18, \
4.230789067417627e-6, -1.7538575991049364e-7, 2.46291003767111e-18, \
-2.675991666328335e-10, -1.7538575989715213e-7, 9.691788331813837e-9, \
-0.000025750975332148513, -6.4291945861158105e-18, \
4.25376858084295e-6, -1.753857599234168e-7, 4.254383932972973e-6, \
1.823775065384436e-17, -1.7302864183515451e-19, \
-9.49767071453172e-18, -2.2796746591327704e-18, \
1.2424392447688641e-17, -1.1822708926825353e-17, \
1.5457374087771653e-17, -1.7687783261122925e-7, \
-1.759790182761045e-17, 1.514425138973801e-17, 7.889870128789842e-18, \
9.317078976169822e-18, 2.5528966214930684e-18, \
4.9359684687078465e-19, 1.2636657472296742e-18, \
1.1080728597821124e-18, 1.917969616689785e-17, \
1.0185544179246604e-17, 3.94399355779401e-18, 6.7341241045171136e-18, \
2.46283516567701e-18, 1.8386631702318604e-17, 3.522351866426998e-18, \
-4.166932304730537e-10, -2.0484072287152985e-17, \
1.9293678392888287e-17, 1.4578987903730773e-17, \
1.4234489168237666e-6, -9.73840023470124e-18, -2.350900788197189e-7, \
9.691788307932118e-9, -2.365681685958132e-7, -2.1384808283479206e-17, \
8.490746322316736e-18, -1.1311757134543116e-17, \
1.8456723159399288e-18, 3.142518598650507e-18, 7.731360556782526e-18, \
2.2677419981973458e-17, 1.3508407605454277e-8, \
2.1323011240340474e-18, 1.5587900042265805e-17, \
7.214205141603654e-18, 0.023087514789948443, 2.08904934554477e-17, \
-0.0038156294832041283, 0.00015821781141705943, \
-0.003758302156981521, 4.650405269266426e-19, 1.4077435717507677e-17, \
-1.114246039242246e-17, -0.0037598014822432147, \
-6.827934829297187e-18, 0.0006306023268302205, \
-0.000026286230363465332, -1.0387811176324156e-17, \
-4.0266722945558996e-8, -0.0000261496152121619, \
1.4541114252302066e-6, -0.003759745195512087, \
-1.1608043575792155e-17, 0.0006211270496212049, \
-0.000025750975332153077, 0.0006210711239844106, \
1.4525863886935728e-17, -7.314204720272018e-18, \
-6.137012234350199e-18, 6.490068199030867e-18, \
-4.690086802758893e-18, 2.4046027386460313e-20, \
-2.912282967600061e-18, -0.00002561396957340032, \
1.391563684140839e-17, 7.801475022555779e-18, 1.262080121021377e-17, \
-0.0037598014822431912, 1.0185906612202572e-17, \
0.0006213837763661147, -0.000025766311028465675, \
0.0006118057332472522, 7.250545252256594e-18, 6.7835357270062485e-18, \
-2.654943641462156e-18, 0.0006213298458967432, \
1.0284763288784735e-17, -0.00010421997760643073, \
4.344545716597531e-6, 1.3900938044046313e-17, 6.712232756012791e-9, \
4.3445457166101245e-6, -2.4176149029213115e-7, \
-1.3360652987757042e-17, -9.7063536675625e-13, \
-2.2883796115020867e-18, -3.7069687271315354e-18, \
4.15456527307203e-19, 1.7789003053689877e-11, 6.655554216040541e-9, \
-4.2622078076642385e-10, -0.00002562952482520858, \
-1.1977711879741416e-17, 4.321794037310846e-6, -1.806611707868761e-7, \
8.721338566038224e-19, -4.271995159030984e-10, \
-2.4033709416138983e-7, 1.3809685126238895e-8, \
-0.0037583021569815205, -1.0290512255856423e-18, \
0.0006211270496212127, -0.000025755425821069272, \
0.0006118037553734858, -3.1097446610303653e-18, \
-1.0894193632966301e-17, -3.3251403208061824e-18, \
0.0006118057332472607, -1.0501974986817113e-17, \
-0.00010261311534405465, 4.277343357629548e-6, -7.65757758797134e-18, \
6.550282498118248e-9, 4.2543839329490005e-6, -2.365681686041593e-7, \
0.0006210711239844159, -1.0459171697650335e-17, \
-0.00010260346926308543, 4.253768580872035e-6, \
-0.00010260346926310417, -1.3032422920784748e-18, \
-2.176066284035657e-18, 2.7286413548113688e-18, \
-3.619435673592105e-18, -1.1577061074416727e-17, \
2.1971201760146042e-17, 2.2395549732019594e-17, 4.253768580871824e-6, \
-1.0478574386138632e-17, 9.654705659925047e-18, \
2.1000074933251593e-17, 8.7960755155298e-19, 4.925564450394392e-19, \
1.2047656834353502e-17, -8.428335842090295e-18, \
1.427413911088621e-17, -1.02577753009926e-18, \
-1.8460561312019618e-17, -1.0241239654008221e-17, \
3.565334036435882e-18, 8.543960479812327e-18, 6.00170664487341e-18, \
-6.728081634024865e-18, 6.491776929769759e-9, 2.2525367301837652e-17, \
1.2927409526247903e-17, -1.2604997978577222e-17, \
-0.000025613969573389654, 9.857156108106953e-18, \
4.230789067422362e-6, -1.753857599284488e-7, 4.253768580864438e-6, \
1.7112251201886807e-17, 1.0088699574669874e-17, \
-1.719631069091311e-18, -4.905714049320633e-18, \
1.581877642773412e-17, -2.1817340034063246e-17, \
1.6811353597211604e-17, -2.3509007881054276e-7, \
-2.0560304932842335e-17, -1.2578375944263156e-17, \
-1.9478713047373143e-17, -0.0038156294832041486, \
-1.0477369839913887e-17, 0.0006306023268302199, \
-0.000026149615212171713, 0.0006211270496212055, \
-4.308661584052829e-18, 5.9152291767646025e-18, \
1.1839617536758587e-17, 0.000621383776366118, \
-1.1781683745088816e-18, -0.00010421997760642405, \
4.3445457166006055e-6, 5.67077447002541e-18, 6.655554225275479e-9, \
4.321794037323327e-6, -2.40337094159311e-7, 0.0006211270496212102, \
-5.732858905836046e-18, -0.00010261311534406506, \
4.254383932962545e-6, -0.0001026034692631052, 2.6581088601486664e-18, \
-1.7983657576036354e-17, 2.123110040896227e-17, \
6.548311239522684e-18, 3.1471766986637105e-18, 6.634359804313511e-18, \
5.371422000122179e-18, 4.230789067420144e-6, 1.0404755615118247e-17, \
6.825511693176797e-18, 1.445865063646505e-17, 0.0006306023268302163, \
6.682015939134223e-18, -0.00010421997760641277, 4.321794037308569e-6, \
-0.00010261311534406179, 1.8366616646140872e-17, \
1.9755010578079843e-17, 6.383378433769853e-18, \
-0.00010421997760641123, 1.6853941317251102e-17, \
0.000017481570767978864, -7.287773338960526e-7, \
-1.1609284604588873e-17, -1.1260840629151857e-9, \
-7.287773338954507e-7, 4.055693179079035e-8, 1.4854507213551635e-17, \
2.1703352819398128e-13, 2.0087791564633542e-17, \
2.770749593562995e-18, 1.2819999287412091e-17, \
-3.0216752201931882e-12, -1.1260840438985973e-9, \
7.213670366082854e-11, 4.3217940373307496e-6, \
-1.3952641288541996e-17, -7.287773338997552e-7, \
3.0466385355202495e-8, -1.3792376204035339e-17, \
7.213668328783506e-11, 4.055693179123023e-8, -2.3308129723156825e-9, \
-2.9672184621793324e-18, -1.0661811015954079e-10, \
-3.658607905773869e-18, 7.538960171556246e-18, \
-1.1169836473657488e-18, 1.7788983479177607e-11, \
2.1700346147469897e-13, -1.1811594678080298e-12, \
-7.163309308794289e-18, 1.733724902680185e-11, \
-1.5517037594820413e-17, 2.203870188336977e-17, \
-1.199816130461489e-17, 2.134157897778198e-17, -1.50749236301398e-17, \
-1.3895773983874558e-17, -6.880214060293026e-18, \
1.7788992885993655e-11, -2.0299516888060383e-17, \
-1.461691100110604e-18, 3.5882134533900705e-18, \
-3.0216807211120275e-12, -3.021720214928289e-12, \
3.9887557917166164e-13, 6.655554241637643e-9, 5.899351646178542e-18, \
-1.1260840750947858e-9, 4.717936146900203e-11, \
-1.4904940219007475e-17, 2.687271234659165e-13, \
7.213667090761538e-11, -4.2562949134349765e-12, \
-0.000026149615212162335, -1.4542510147630474e-17, \
4.321794037327339e-6, -1.7922180368012814e-7, 4.254383932958409e-6, \
-2.1900442434733044e-17, -1.7179693021440632e-17, \
2.411375229114263e-18, 4.3445457165867345e-6, 3.2111474920901257e-18, \
-7.287773339021633e-7, 3.038318624760945e-8, -1.9147074069936805e-17, \
4.7179391509092524e-11, 3.046638535995402e-8, -1.6962967507474187e-9, \
1.4130215753570244e-18, -1.1811389377017714e-12, \
-1.0150437033118937e-17, 1.5010500598296728e-19, \
2.32903470191149e-17, 3.988342352077882e-13, 7.21367021490066e-11, \
-4.676564917045398e-12, -2.40337094161491e-7, 5.615745067109827e-18, \
4.0556931780349847e-8, -1.696296755013055e-9, 1.1282397270392785e-18, \
-4.256290480218778e-12, -2.3308129889291386e-9, \
1.346870732839696e-10, -1.8627330511130018e-17, \
2.4156521359708735e-7, 6.494821692479772e-19, \
-1.7124589369171783e-17, 1.2352220423268595e-17, \
-4.026672294537481e-8, -1.0661811116174822e-10, 2.562141387535213e-9, \
1.2428113826700561e-18, -3.931119748526437e-8, \
-6.418142631221596e-18, -6.532094107075111e-18, \
8.991683265330562e-19, -6.299744128297728e-18, \
1.4519342349912287e-18, -9.82367003733667e-18, 3.998685638445046e-19, \
-3.934046357050563e-8, 2.2010749020852883e-18, \
3.0838266325288642e-18, -1.0395404885649757e-18, \
6.655554217547312e-9, 1.7788980698197267e-11, -4.262207618211487e-10, \
-9.706429691985755e-13, -2.873600343388672e-18, \
2.1700612825513402e-13, -8.366416318480085e-13, \
-4.450523502710719e-18, -2.773524925210234e-10, \
-1.181149023125797e-12, 2.4921703967930955e-11, \
2.7090400615291258e-18, -3.9311197482812976e-8, \
-8.92464279055262e-18, -6.256972934187322e-18, \
2.2287952061067944e-18, 6.550282510956526e-9, 1.733726359796043e-11, \
-4.1669325049301426e-10, 3.100208182103863e-18, 6.491776941500678e-9, \
1.3956059955089992e-17, -2.5613246996020725e-18, \
1.5076650871038135e-17, -2.3459456250951748e-17, \
6.016863569195066e-18, -8.964486693573157e-18, \
3.4224286846022513e-18, -7.299254005624304e-18, \
1.4057199165658915e-17, -1.0052478850411888e-17, \
1.4239984613479648e-17, -1.2225228944239988e-17, \
-2.188281333100946e-17, -2.075241202609933e-17, \
5.242256838739691e-18, -2.675991708457973e-10, \
2.9586707166900424e-18, -2.2212384182630438e-17, \
-8.019811654106106e-18, -1.5957583548970155e-17, \
1.624064214044138e-17, -7.230085932827122e-18, 5.944999603190435e-18, \
-4.026672294440226e-8, -4.333210131635999e-18, 5.658480150678273e-18, \
-2.03085216291846e-17, 6.7122327650936355e-9, 1.778898547299619e-11, \
-4.271995412896422e-10, 2.2276571779280073e-18, 6.550282500856171e-9, \
1.4009868075001578e-17, 1.1265894597800201e-18, \
-6.7421559898606854e-18, -3.6496377094607446e-18, \
2.9588579327866806e-18, -1.6820149847264248e-17, \
2.9234777079335408e-18, 6.655554234928583e-9, -8.423779544421623e-18, \
2.1230415301751364e-17, -7.527388443205901e-19, \
-1.1260840834144958e-9, -3.021681897296896e-12, 7.21366930544935e-11, \
1.7788968905685733e-11, -1.436122693977631e-17, \
-3.0216935005415915e-12, 2.6877037964246525e-13, \
2.037589789057141e-17, 4.7179393792392133e-11, \
3.9883235786566067e-13, -4.256308606866826e-12, \
-4.026672293765198e-8, -1.6420347220293999e-18, \
6.6555542459690855e-9, -2.773525096016856e-10, 6.550282499794865e-9, \
1.2322761421517872e-17, 9.98910261410729e-18, \
-1.5957007998772325e-18, 6.712232767332052e-9, 5.274358884800174e-18, \
-1.1260840306888678e-9, 4.717935955824996e-11, \
2.0790737409870555e-18, 1.4771617215063957e-13, \
4.717938063245055e-11, -2.65079977296332e-12, -8.300893697419158e-19, \
-2.7735249520424953e-10, 2.616866548616658e-18, \
1.8758818271056653e-17, -2.0674993200862035e-17, \
4.717935090487466e-11, 2.6877140991836687e-13, \
-3.042912376144223e-12, -4.2719953981127544e-10, \
-1.1627644392619342e-17, 7.213665070384783e-11, \
-3.0429157734701683e-12, -3.333477757862517e-18, \
-2.650844400129694e-12, -4.2563117186521395e-12, \
4.957275222192655e-13, 0.00015821781141704515, \
5.6077452359846585e-18, -0.00002614961521215829, \
1.087372310617577e-6, -0.00002575542582106623, \
-1.4724092465830362e-17, 1.2644968698376723e-17, \
1.0514582142282001e-17, -0.00002576631102846372, \
-1.642003213422201e-18, 4.321794037327329e-6, -1.8066117075944795e-7, \
7.096027201414734e-18, -2.77352485143314e-10, -1.7922180368530412e-7, \
9.998581550069414e-9, -0.000025750975332148245, \
-1.5834793329620793e-17, 4.254383932968444e-6, \
-1.7687783261617547e-7, 4.253768580863549e-6, \
-2.6864300684515074e-17, 2.5680882791585507e-18, \
7.415524929493135e-18, -8.345044002343626e-18, -1.0038041434895e-17, \
-5.250886506445947e-18, -2.980373360681748e-20, \
-1.7538575991865398e-7, -2.187213038633592e-17, \
-1.92501509075868e-17, -1.2138366759477323e-18, \
-0.000026286230363447663, 5.8747425094415165e-18, \
4.34454571662461e-6, -1.806611707734891e-7, 4.277343357633084e-6, \
-1.7486075992103216e-17, 4.553743231671736e-18, \
-1.6474968562261122e-18, 4.344545716607865e-6, \
-4.355726442454192e-18, -7.287773338936858e-7, 3.046638536316292e-8, \
1.8863219187113164e-17, 4.717936978793773e-11, 3.038318624659041e-8, \
-1.6962967692056437e-9, -7.374107823318405e-19, \
-8.366341120837203e-13, -1.6756509602317995e-17, \
-1.9337439989467818e-17, -1.2609666249062882e-17, \
2.687692177637416e-13, 4.717940178699724e-11, \
-3.0429043420945375e-12, -1.8066117075779484e-7, \
1.4403849109022345e-17, 3.046638535808995e-8, -1.2772231372590204e-9, \
-1.8299060849603614e-18, -3.0428876103976336e-12, \
-1.696296770417311e-9, 9.781788703476468e-11, 3.747040281795467e-18, \
2.5621413878517915e-9, -9.298762624522534e-19, \
1.0392186652475347e-18, -4.9059392034529625e-18, \
-4.2719954655427503e-10, -1.181152583895277e-12, \
2.7435236009649263e-11, -3.25365458054998e-18, \
-4.1669324710044786e-10, -1.5969342395738515e-17, \
1.7259672788304634e-17, -4.16635916136378e-18, \
-2.1337878637081302e-17, 1.760717941434813e-17, \
1.9812550385534177e-17, 1.9012609777093236e-18, \
-4.262207751814303e-10, -1.4271495378323156e-17, \
6.5895727200736215e-18, 7.536882730094569e-18, 7.21366755071039e-11, \
3.988384206455678e-13, -4.676548692241923e-12, \
-4.262207889099058e-10, -1.940942759496826e-18, \
7.213669393596606e-11, -3.042896646190339e-12, \
-1.7184960554145182e-17, -3.04286064090671e-12, \
-4.676577556396756e-12, 5.521745592962808e-13, 1.4541114252323236e-6, \
-3.81470851803743e-18, -2.403370941354954e-7, 9.998581547280907e-9, \
-2.3656816858985853e-7, 1.6958168966059735e-17, \
-1.054939884666123e-17, -9.324652146476768e-18, \
-2.4176149028414633e-7, 1.837253861349094e-17, 4.055693175163809e-8, \
-1.696296742551168e-9, 4.529995087762043e-18, \
-2.6508469059547682e-12, -1.6962967749296384e-9, \
9.48060297033531e-11, -2.0860746989022536e-18, \
2.4921713235207127e-11, -7.639025702868462e-18, \
1.5040347686677828e-17, -9.736674389244427e-18, \
-4.256281198833657e-12, -4.256282338248827e-12, \
5.522086356983485e-13, 1.3809685122257332e-8, \
-1.2771469314625562e-17, -2.33081301058182e-9, 9.781789475592459e-11, \
2.0225617899652904e-17, 4.95777311111952e-13, 1.3468704570171075e-10, \
-7.845366158006227e-12, 4.403100572098428e-17, \
-0.0009523338813057225, -3.576096549692103e-18, \
-7.500870186055743e-18, 2.913831735123643e-17, \
0.00015821781141707244, 2.4156521358667187e-7, -8.746122967217227e-6, \
-4.9072075172359104e-17, 0.0001549986996418619, \
9.740800287091388e-18, 8.230505339769111e-18, 1.2256746780528019e-17, \
-8.614117347003027e-19, -7.985275326046757e-18, \
5.046640064698156e-19, -5.414946161848134e-17, \
0.00015508867446751065, 3.7295997981293e-18, 1.4184932079520267e-18, \
9.159498678204276e-18, -0.000026149615212167722, \
-4.026672294061062e-8, 1.4541114252411175e-6, 5.786086754711174e-12, \
-6.725223354562508e-19, -1.0661810597341323e-10, \
2.562141390817437e-9, -1.2694358713650005e-17, 1.0873723106118088e-6, \
2.5621413978151733e-9, -8.304944942012447e-8, 1.1729632081728484e-17, \
0.00015508514031020555, -4.929619341391194e-19, \
3.100025839031118e-18, -1.4026523417683688e-17, \
-0.000025755425821081622, -3.9311197482675737e-8, \
1.4234489168271205e-6, 1.0676390598302626e-17, \
-0.00002561396957339933, -1.206351333069293e-18, \
-4.409430961062702e-18, -1.6354719376794807e-18, \
-1.6494076790256215e-18, 9.796642117446524e-18, \
3.306741096995196e-18, 1.6349836813943674e-17, \
-8.063635495885623e-18, 1.749295302523412e-18, \
1.6314049394524549e-18, -1.5646833920620546e-18, \
-4.1916929491524533e-19, -3.678354512878438e-18, \
5.533726898812077e-18, 8.073924336886498e-18, 1.0562677716578719e-6, \
8.472291215029328e-18, -6.97614142658298e-19, 6.974925820787984e-19, \
4.545575141370685e-18, 1.2336129323834678e-17, 8.848783491742406e-18, \
2.1934487905227542e-17, 0.0001550886744675152, \
-3.2555190853914217e-18, 1.1103544613918878e-17, \
-8.261423500422173e-18, -0.000025766311028463415, \
-3.9340463563811314e-8, 1.4243496326077352e-6, \
1.5131946099742843e-18, -0.000025232079350491443, \
6.418184449820878e-18, 3.0301436824606817e-18, \
-7.389503139058286e-18, -9.179482151868622e-18, \
-4.805457599009283e-18, -2.157145693031046e-18, \
1.4605720783518704e-17, -0.000025629524825205608, \
-1.483603687550841e-17, 5.758330309225488e-18, \
1.6232011600332294e-18, 4.321794037331879e-6, 6.655554232385064e-9, \
-2.4033709413442136e-7, -9.706396987407272e-13, \
-1.0166119677833515e-17, 1.7788997041096533e-11, \
-4.2719950961587265e-10, -4.531254691009145e-19, \
-1.8066117076010313e-7, -4.2622077199994133e-10, \
1.3809685150185109e-8, 5.786072181702987e-12, \
-1.1448618848488201e-17, -9.706435439038237e-13, \
1.3558995985422094e-13, -9.40675887270646e-13, \
3.4368188635528717e-18, 1.0545795340692808e-17, \
2.543301568273974e-18, -9.706389560176965e-13, \
-3.815453210010132e-18, 2.1700673220837687e-13, \
-1.1811299586490837e-12, -3.874147521885301e-18, \
-2.7735249819696996e-10, -8.366518940547517e-13, \
2.4921711719649697e-11, -1.366681622951141e-17, 1.057212679593637e-6, \
1.8268849452911977e-18, 6.4510503321711426e-18, \
6.716464424642062e-18, -1.7922180368194224e-7, \
-2.773524942292431e-10, 9.998581538715105e-9, 1.3558677204339235e-13, \
-6.732582656154037e-18, -1.181146131283508e-12, \
2.7435244609209796e-11, 6.2292795311129195e-18, 9.998581535050304e-9, \
2.4921708296012802e-11, -7.930829174936557e-10, \
-3.4508750616557317e-17, 0.00015499869964180624, \
2.206660299640772e-18, 6.064572452572048e-18, 7.31425049409113e-18, \
-0.000025750975332151535, -3.931119748488715e-8, \
1.4234489168304557e-6, 7.643602133178102e-18, \
-0.000025227419007964702, -1.259266620935837e-17, \
-5.393253586958202e-18, -3.04306148351774e-18, \
-6.547784144252104e-18, -3.157161228955562e-19, \
7.808287200514198e-18, 1.167063624332077e-17, \
-0.00002523207935048194, -4.496249643224952e-18, \
-2.899637550333151e-18, 3.748329365200522e-18, 4.2543839329496485e-6, \
6.550282498233071e-9, -2.3656816859680303e-7, -9.406783407047968e-13, \
8.546870124673015e-18, 1.7337273858757064e-11, \
-4.1669324861994276e-10, 8.175893444270184e-18, \
-1.7687783263473537e-7, -4.1669322438482395e-10, \
1.3508407604124389e-8, -3.510813702702375e-18, \
-0.000025613969573395814, -8.397790208653624e-18, \
5.5299329621372375e-18, 2.3380378249190996e-18, 4.253768580852029e-6, \
6.4917769350763916e-9, -2.350900788071928e-7, -4.11893108038756e-18, \
4.230789067452998e-6, -2.6059568653848505e-18, 5.106263528946373e-18, \
1.7386642887542645e-18, -2.278932745515248e-17, \
1.1936369623222396e-17, 1.5592932178762342e-17, 2.42775249146408e-18, \
5.108362382180339e-18, 1.910119049865635e-17, \
-1.3241087281165978e-17, 1.419102901652815e-17, \
-7.204416685001663e-18, -4.5962644089874306e-18, \
7.811951489916645e-18, 5.157262446642965e-18, -1.7538575992185031e-7, \
-5.767663878611424e-18, -9.234437427036272e-18, \
1.0638771388197084e-17, 1.1310689624120169e-17, \
1.706592743254476e-17, 2.237971033369815e-18, 2.0245343640181348e-17, \
3.096939511004278e-18, -1.3328861377296438e-18, \
-4.124004187524489e-18, -3.349967153444078e-18, \
2.4208610787850148e-18, 2.464483727324047e-18, \
-3.1545657927894582e-18, -4.034447624683168e-18, \
-3.384276694213419e-18, 2.5070066176767383e-18, \
-1.939302690282312e-17, -3.626346965269104e-19, \
-1.094349967861246e-17, 1.007759906012538e-17, 2.178380454978167e-18, \
-4.965631580554465e-18, -4.647432790498628e-19, \
7.727762006092798e-18, -1.3099460943189981e-17, \
-9.572205576678565e-18, -4.635497410337826e-18, \
1.1594824215604055e-17, -2.083515023854972e-17, \
-1.0468357131084835e-18, -2.675991586047665e-10, \
1.1766028512861237e-17, -1.3684733581698613e-17, \
-3.776597068934426e-18, 1.991494788282307e-17, \
-1.946975419451196e-17, 5.0259977545627345e-18, \
5.006162282558166e-18, 1.0562677716606114e-6, \
-1.1235203407158571e-17, -8.798836682115763e-18, \
1.7972498010738698e-18, -1.753857598971695e-7, \
-2.675991660555979e-10, 9.691788307453667e-9, 1.1648242636913764e-17, \
-1.7538575991064245e-7, 1.0310242001719412e-17, \
-9.798335470749326e-18, 1.272171365255335e-17, \
-1.1059261719173845e-17, 1.6102853220056374e-17, \
-2.1158178879248662e-17, 2.379553114398815e-18, \
9.954174500016846e-18, 1.3466315365033524e-17, \
-1.761146734427386e-17, 6.98714933136298e-18, 1.3287382995042175e-17, \
2.2315007969939325e-17, 2.1214193309062096e-17, \
1.220575298042248e-17, 9.69178832329791e-9, 6.370738300811246e-18, \
2.3207936445516874e-18, 5.635739962898273e-18, \
-5.297696535271652e-18, 8.588119576726455e-18, -1.53369271471492e-18, \
1.3042614929767184e-17, 0.00015821781141705176, \
-1.0546790201671194e-17, 6.011480115254893e-18, \
-6.601406782382975e-18, -0.000026286230363454443, \
-4.0266722941696565e-8, 1.4541114252385999e-6, 9.929295674719739e-18, \
-0.000025750975332156712, -3.664603642101403e-18, \
-1.5773770456872341e-18, -5.37155594290243e-18, \
5.674671864335553e-18, 7.387641614767547e-19, 1.382211682047721e-18, \
1.0497370562754805e-17, -0.000025766311028460183, \
-5.14774563727172e-18, -2.815747991613979e-18, \
-7.721686457919803e-18, 4.344545716597087e-6, 6.712232755014606e-9, \
-2.4176149029911674e-7, -9.706412239145816e-13, \
9.776078858625864e-18, 1.7788994369753564e-11, \
-4.2622079145350156e-10, -8.042583962013738e-18, \
-1.806611707851493e-7, -4.271995285100956e-10, 1.3809685129565207e-8, \
-1.0867247661837007e-17, -0.000025755425821069364, \
-6.16564641648771e-18, -5.742527188604367e-18, -3.54589661180833e-18, \
4.2773433576251765e-6, 6.550282505555434e-9, -2.3656816860455508e-7, \
-4.210273630249303e-18, 4.253768580878711e-6, \
-1.0939719012545747e-17, -1.805004406531674e-17, \
3.5442632143047246e-18, -6.13169230944418e-18, \
1.7857922588734517e-17, 1.8062959969640988e-17, \
-5.133244789594109e-18, -5.0146751746659484e-18, \
5.28310293677633e-18, 8.238335150884248e-18, -1.6469138392105533e-17, \
1.3666225890433115e-17, -1.0675808263428965e-17, \
-1.5132268467078503e-17, 5.318188145816227e-18, \
-1.753857599037266e-7, 2.6274820265305904e-17, \
-5.980603342097524e-18, -1.8402936181106003e-17, \
-2.062978436996236e-17, 8.030657287079616e-18, \
-2.382504148123852e-17, -5.515041475318495e-18, \
-0.000026149615212168718, 8.63451359788033e-18, \
-5.251759951040888e-19, -4.784026934328176e-18, 4.344545716611851e-6, \
6.655554220225933e-9, -2.4033709415293046e-7, 3.631777425346714e-18, \
4.254383932939889e-6, 1.8782508402577496e-17, -4.393968290345175e-22, \
-1.897737578551627e-17, 9.864333965007848e-19, \
-2.0620911471494188e-17, -5.001490602423495e-18, \
1.2999168794609618e-18, 4.3217940373332864e-6, \
-1.0661338664386209e-17, -7.565615260369349e-19, \
1.343913786120072e-17, -7.287773339262319e-7, -1.1260840864569583e-9, \
4.055693174408363e-8, 2.1701263976212298e-13, -9.80028272465866e-18, \
-3.0217005092093974e-12, 7.213668858079535e-11, \
-7.355321452076582e-18, 3.0466385316987585e-8, 7.213667287837028e-11, \
-2.3308130046427314e-9, -1.066181112991832e-10, \
7.496162023607415e-19, 1.7788972741293763e-11, \
-1.1811354824430705e-12, 1.7337269167442034e-11, \
-1.825654361700131e-19, 1.1102901657635269e-17, \
6.009275305585215e-18, 1.7788993851816337e-11, \
-2.3738753687632196e-17, -3.021688750343837e-12, \
3.9886288787868734e-13, 1.283788133425984e-17, \
4.7179404978914944e-11, 2.6872459892740304e-13, \
-4.256305773188109e-12, 5.34781506286342e-18, -1.7922180368282512e-7, \
1.9537552088096413e-17, -1.443396579379331e-17, \
1.9809258897409913e-17, 3.038318624503929e-8, 4.717936683397346e-11, \
-1.6962967692237534e-9, -1.181131172887774e-12, \
1.2136231145405254e-17, 3.9886692966701894e-13, \
-4.676552182920618e-12, 1.0875626248982153e-17, \
-1.696296765215228e-9, -4.256270700024109e-12, \
1.3468704328915575e-10, 2.415652136019629e-7, -8.349159837758335e-20, \
-4.026672294902419e-8, 2.5621413993011958e-9, -3.931119748579088e-8, \
-9.14667603413029e-18, 6.693339726498436e-18, -9.850520024479719e-18, \
-3.9340463573750484e-8, 5.63475052268484e-18, 6.6555542130813985e-9, \
-4.262207698581829e-10, -7.691924995767284e-18, \
-8.366620621674519e-13, -2.773524924621341e-10, \
2.4921683930114682e-11, -3.931119748445467e-8, \
1.2317573010572325e-17, 6.5502825239307436e-9, \
-4.166932193452269e-10, 6.491776949615145e-9, 1.0213095466531037e-17, \
-8.299669435587756e-19, -1.8163990007305122e-17, \
1.1307145017889096e-17, 1.6829516382281637e-17, \
2.0500600815847615e-17, 1.7703391172166866e-17, \
-2.6759915745017075e-10, -9.088077466843307e-18, \
-2.1700044661675443e-18, 1.1411539333656594e-17, \
-4.026672295166743e-8, 9.182276504809048e-18, 6.712232747851612e-9, \
-4.2719952142559576e-10, 6.550282501858092e-9, \
-2.342816673162113e-17, 2.4583703213772674e-17, \
-3.2044369932687174e-18, 6.655554216749107e-9, 4.725036480436937e-18, \
-1.1260840526569752e-9, 7.213669726774214e-11, \
-1.9644395778941843e-18, 2.687637155140955e-13, \
4.717936534591336e-11, -4.256318737098312e-12, \
-5.614542605391131e-18, -2.773525144945568e-10, \
-1.3571003588287643e-17, -3.748477453200875e-18, \
2.197877447503541e-17, 4.717940195330177e-11, 1.477176896747689e-13, \
-2.6508026859225164e-12, -2.773524874475652e-10, \
-1.452305137805687e-17, 4.7179389260511365e-11, \
-3.042884712526406e-12, -6.392901983812667e-18, \
-3.0428595117438684e-12, -2.6508219240027575e-12, \
4.957394154065988e-13, 4.747894022763099e-20, 1.0873723106082341e-6, \
-5.701631362862171e-18, 1.995771308445722e-18, 7.260454015715063e-18, \
-1.8066117077810266e-7, -2.773524846947588e-10, 9.998581547074276e-9, \
-2.3626236146353113e-18, -1.76877832643623e-7, \
-2.1462342380549448e-17, 3.169264124431727e-18, \
-6.285600647547033e-18, -1.3278072481927527e-17, \
2.1568451265914606e-17, 5.885168735684663e-18, 9.537497568640454e-18, \
-1.806611707661538e-7, -1.0302434881294271e-17, \
4.039586695720318e-18, 1.6089283074610702e-17, 3.046638535796668e-8, \
4.717939494518093e-11, -1.6962967600475954e-9, \
-8.366553922327303e-13, -1.9670771736386903e-17, \
2.6876025164169745e-13, -3.042863049045361e-12, \
-1.2052846931244549e-17, -1.2772231384953316e-9, \
-3.04287428003978e-12, 9.781791868050494e-11, 2.562141392233622e-9, \
5.376250085878577e-18, -4.2719954321442226e-10, \
2.7435240708499494e-11, -4.1669322975686277e-10, \
-9.328694377626565e-18, -1.281135289022986e-17, \
-2.303306310842851e-17, -4.2622077913826305e-10, \
7.080598296388847e-18, 7.213668609086683e-11, -4.676583269034692e-12, \
-5.477376030934669e-19, -3.042900619841795e-12, \
-3.042914434134857e-12, 5.521633493595395e-13, \
-4.7733445607709365e-18, 9.998581546928153e-9, \
5.7319852961566536e-18, -1.4070842847670423e-17, \
4.1737965546965185e-18, -1.6962967834295348e-9, \
-2.650800578248797e-12, 9.480598037936734e-11, \
2.4921690037594977e-11, 1.7003334436247065e-17, \
-4.256305910386198e-12, 5.522090447872541e-13, \
-2.0496812009566813e-17, 9.781788719653665e-11, \
4.957484032394062e-13, -7.845360700754663e-12, \
-0.0009523338813057294, 3.9728451174146533e-17, \
0.00015821781141704734, -8.746122967228338e-6, \
0.00015499869964181827, -2.2354427870520383e-18, \
-1.2745892802301558e-17, -1.9905954294171916e-18, \
0.0001550886744675018, -2.8467054313366725e-18, \
-0.00002614961521216056, 1.4541114252351296e-6, \
-8.199948190331791e-18, 2.5621413928950835e-9, 1.0873723106177459e-6, \
-8.304944942639343e-8, 0.00015508514031018988, 3.11094555165113e-18, \
-0.000025755425821068652, 1.423448916829424e-6, \
-0.000025613969573398328, -8.962247816128183e-18, \
-8.646600563866541e-18, -1.8327295811920986e-18, \
-6.003142794718404e-18, -2.2343800479661145e-18, \
7.751122027632297e-18, -7.77112610434559e-18, 1.0562677716545615e-6, \
1.533970483345067e-18, 1.6099860759254701e-18, \
-1.155903642601522e-18, 0.0001550886744675028, \
-1.157106815321644e-17, -0.000025766311028452688, \
1.4243496326128234e-6, -0.000025232079350479256, \
8.491585569489702e-18, -5.460287785683077e-18, \
-9.223648747003544e-19, -0.000025629524825207722, \
4.05644866770471e-18, 4.321794037317977e-6, -2.40337094147143e-7, \
-7.933338543120462e-18, -4.2719952738581304e-10, \
-1.8066117076994084e-7, 1.3809685136387844e-8, \
1.8359372585810643e-19, 1.355856680383211e-13, \
-1.1316735248524309e-17, -8.078891310113616e-18, \
9.445588122380853e-18, -1.1811370428486256e-12, \
-2.773524980269581e-10, 2.492168308412048e-11, 1.0572126795992082e-6, \
-5.043854157153155e-18, -1.7922180367175096e-7, 9.99858153200502e-9, \
2.7239224683964104e-19, 2.7435230132423816e-11, 9.998581537257066e-9, \
-7.930829126932477e-10, 0.0001549986996418265, \
-6.504513528172948e-19, -0.00002575097533215244, \
1.423448916831028e-6, -0.00002522741900796881, 4.72083233999069e-19, \
-9.14098529004565e-18, 1.4394637966653206e-18, \
-0.00002523207935048215, 1.0525194876084351e-18, \
4.254383932953047e-6, -2.3656816857714618e-7, 2.5044229340024625e-18, \
-4.1669322808846786e-10, -1.7687783260784752e-7, \
1.3508407616208606e-8, -0.000025613969573398738, \
-4.0815667412808525e-18, 4.2537685808604675e-6, \
-2.3509007882006799e-7, 4.23078906743399e-6, -2.012978363937815e-17, \
-2.841917398648992e-17, -2.1822217691292768e-17, \
1.0266729433027284e-18, 5.21533308745961e-18, 4.968804864046804e-18, \
-7.336624624233252e-18, -1.7538575991099244e-7, \
-9.646005713675052e-18, -1.748033698490945e-17, \
-2.366139428697071e-17, -1.9646054920509348e-18, \
2.3462550366110126e-19, 8.914988081683456e-19, 8.51126506903491e-18, \
-4.9302823773035145e-18, 3.8818056335100275e-18, \
-2.0419495814802326e-18, -7.945773111498308e-18, \
3.9772067975995774e-18, -9.000876688126421e-19, \
-4.935010073247836e-19, 9.2962563994386e-18, -2.675991833865725e-10, \
-2.062958033103628e-17, 1.1568576167460243e-17, \
2.348130220928363e-17, 1.0562677716532916e-6, 1.26817974116879e-17, \
-1.7538575991420274e-7, 9.691788307089967e-9, -1.7538575991803546e-7, \
1.7958122998548893e-17, 7.877614717925344e-18, \
5.4087459829979585e-18, 3.052018881072297e-19, 6.330218522132021e-18, \
-1.9226647122606823e-17, 1.8842304734179586e-17, \
9.691788324043317e-9, 1.0088300835298083e-17, \
-1.6058424192629055e-17, -1.597821981963996e-17, \
0.0001582178114170848, 3.0989014945495753e-18, \
-0.000026286230363450292, 1.4541114252407073e-6, \
-0.000025750975332150624, -1.7122175189335137e-18, \
2.808774845558516e-18, -3.472478275044518e-18, \
-0.000025766311028459454, 1.6372271366255842e-18, \
4.344545716619691e-6, -2.4176149031446677e-7, \
-2.6982090724871257e-18, -4.262207819405274e-10, \
-1.8066117075676003e-7, 1.3809685141137273e-8, \
-0.00002575542582107232, -1.8515533161260266e-17, \
4.277343357641876e-6, -2.3656816858151203e-7, 4.253768580873263e-6, \
-2.0142907799346286e-17, 1.464145555218906e-18, \
2.0283617994652955e-17, -5.941763143497416e-20, \
-1.1778503624432092e-17, -5.135649971425438e-18, \
1.828362208914294e-18, -1.7538575992128174e-7, 1.388911941577454e-17, \
1.9900792267605247e-17, -1.7189319741230023e-17, \
-0.00002614961521216559, -3.618603603581489e-18, \
4.344545716615761e-6, -2.403370941400691e-7, 4.254383932971316e-6, \
1.1206121083391068e-17, 1.5280950192972748e-17, \
-1.0230962670817148e-17, 4.321794037315089e-6, \
1.6320331987180915e-17, -7.287773339263895e-7, 4.055693177956291e-8, \
-3.73126700916225e-18, 7.21366802208526e-11, 3.0466385356742865e-8, \
-2.3308129926790416e-9, 7.27921335093448e-18, \
-1.1811446077207192e-12, -7.572793671547217e-18, \
1.6317900169290536e-18, 1.9438936398406688e-18, \
3.9885380850479897e-13, 4.7179358095769986e-11, \
-4.256292469703951e-12, -1.7922180366957236e-7, \
1.2244609135499637e-17, 3.0383186256492285e-8, \
-1.6962967675459188e-9, 1.4858283940367396e-17, \
-4.676559934982933e-12, -1.6962967940734526e-9, \
1.3468705658697065e-10, -1.2542732356615359e-17, \
2.562141395067462e-9, -1.7650895382612446e-18, \
1.4735318795678824e-18, -2.167707654514219e-18, \
-4.2622077694557077e-10, -8.366395014561457e-13, \
2.4921683955645063e-11, 6.434851451241346e-18, \
-4.166932299178891e-10, 1.8693374354170084e-18, 7.0949418178133e-18, \
1.3361594826413323e-17, 2.0760831983044746e-17, \
1.7069698779486183e-17, 1.5290509791970072e-17, \
-2.6196049859737667e-18, -4.2719952190636466e-10, \
-1.7175971488534224e-17, 1.710481997753991e-17, \
1.801642080078345e-17, 7.213669078151838e-11, 2.687413506153193e-13, \
-4.2563106042216726e-12, -2.7735250704243236e-10, \
-8.129038492174182e-19, 4.717935033370051e-11, \
-2.650846439447597e-12, 6.849854215578513e-18, \
-3.0429025865157086e-12, -3.0428904179275052e-12, \
4.957848969003827e-13, 1.0873723106104836e-6, 5.7219210690453274e-18, \
-1.8066117076121386e-7, 9.998581551617354e-9, -1.768778326285574e-7, \
1.9706451850542516e-17, 2.16999129180595e-17, 1.3744518683847933e-17, \
-1.8066117078488554e-7, -1.3003770613735998e-17, \
3.0466385344076466e-8, -1.6962967832227393e-9, \
-1.492462788723929e-17, -3.042895464441126e-12, \
-1.2772231361433734e-9, 9.781791683480899e-11, \
1.9405223177633595e-18, 2.7435238414885214e-11, \
1.213360140471163e-17, 2.3963058528273112e-17, 8.028660684308582e-18, \
-4.67654914606867e-12, -3.0428910279627287e-12, \
5.521902113065354e-13, 9.9985815462335e-9, 1.3689919945335988e-17, \
-1.6962967441267304e-9, 9.480602352336402e-11, \
-1.4726037642896258e-17, 5.521682571196836e-13, \
9.781789809645964e-11, -7.845333873190673e-12, \
-3.177508292743222e-17, -8.746122967205592e-6, 4.456449569052013e-18, \
7.706956397161978e-18, -1.7303916997858162e-18, \
1.4541114252345655e-6, 2.5621413992236906e-9, -8.304944942755166e-8, \
4.767472269436102e-18, 1.4234489168228387e-6, -1.387681492359617e-18, \
-6.818851909714598e-18, 1.1235905982489431e-18, \
1.0453512103028784e-17, 7.655114695029703e-18, \
1.2100914274584535e-17, 1.2573792361017722e-18, 1.424349632600658e-6, \
-1.2265560676584443e-17, -6.452400034238621e-18, \
5.1948278396674936e-18, -2.4033709416250146e-7, \
-4.271995237653911e-10, 1.3809685153844155e-8, \
1.3558306655362595e-13, 3.339195508147678e-18, \
-1.1811452412752167e-12, 2.4921686835209296e-11, \
-2.1957305033882334e-18, 9.998581557865257e-9, \
2.7435211513149958e-11, -7.930829095956601e-10, \
1.3624424994857352e-17, 1.4234489168245366e-6, \
-8.786130042124376e-18, 2.0846457412668222e-18, \
7.220774267309015e-18, -2.365681685841785e-7, \
-4.1669322054405386e-10, 1.3508407598234218e-8, \
8.155867877215447e-18, -2.3509007881849322e-7, \
1.3131111138195481e-17, -6.378988604456223e-18, \
1.021102976868383e-17, 5.0020108595887524e-18, \
-6.231497834123376e-18, 8.353625678978225e-18, 7.03261226450537e-19, \
-6.502125856107718e-19, 3.97776389012577e-18, \
-1.1567115779855395e-17, 2.1372334056414928e-18, \
-6.207331448552389e-19, 4.709204582042737e-18, 1.731560876678288e-17, \
4.3154068966032375e-18, 9.691788302195865e-9, \
-1.1486815288247701e-17, -3.1723540300991634e-18, 4.412999735414e-18, \
5.6364707910844715e-18, -2.1399064631510168e-17, \
6.502002749844297e-18, 2.149457281249623e-18, 1.4541114252375794e-6, \
-1.4427987437756502e-17, -1.442135203388097e-17, \
-2.0148239363938283e-18, -2.4176149029173617e-7, \
-4.262207609462294e-10, 1.3809685131564922e-8, \
-6.638593015296419e-18, -2.3656816859660075e-7, \
2.836589373312334e-19, -8.301191219050286e-19, 9.55524929734109e-18, \
1.8385860919287763e-17, -1.8629961010478343e-17, \
1.9460832626648525e-17, 9.400758269960117e-18, \
-2.4033709413928995e-7, -2.718892662332475e-17, \
-2.2627773249497212e-17, 2.0278098486443273e-17, \
4.055693175199889e-8, 7.213668801491085e-11, -2.330812979223743e-9, \
-1.1811476154806285e-12, -2.115276982178803e-17, \
3.988758914104486e-13, -4.256306497427372e-12, \
2.1118585856838458e-17, -1.6962967576772287e-9, \
-4.676567780298428e-12, 1.3468708061682263e-10, 2.562141393060802e-9, \
-7.718942699862173e-18, -4.262207698259613e-10, \
2.492171061899235e-11, -4.166932406140889e-10, \
-2.102152896873068e-17, 3.5474318604118816e-18, \
1.3567518461749545e-17, -4.2719954702875855e-10, \
2.7224365263810784e-18, 7.21366762693623e-11, -4.256309608059045e-12, \
1.259265678491254e-18, -2.6508158636363235e-12, \
-3.0428716974859333e-12, 4.957572492580823e-13, \
1.1415582679024688e-18, 9.998581560525937e-9, 6.985424966052103e-19, \
-3.121282899814337e-18, 2.1238442293108896e-17, \
-1.6962967932877265e-9, -3.042869466814943e-12, \
9.781790088221128e-11, 2.743523618135107e-11, 2.577895818154352e-18, \
-4.6765473081262045e-12, 5.521862153323019e-13, \
-3.9382636127697635e-18, 9.480600940297361e-11, \
5.521790747929279e-13, -7.845379794296696e-12, -8.746122967202756e-6, \
-1.0099346409490245e-17, 1.4541114252354097e-6, \
-8.304944942589343e-8, 1.4234489168264308e-6, 8.116758680906254e-18, \
-2.2956877674040667e-19, 5.713723542005311e-18, 1.424349632601616e-6, \
2.7946181279885155e-18, -2.403370941570215e-7, 1.380968512486378e-8, \
6.524251285325134e-19, 2.4921702989394685e-11, 9.998581533298584e-9, \
-7.93082916560925e-10, 1.4234489168263675e-6, -1.000623371024358e-18, \
-2.3656816858488547e-7, 1.350840760399339e-8, -2.3509007880237378e-7, \
-1.0095086198698878e-17, 3.762496755466318e-18, \
-1.6819497026482198e-17, 5.842380404425237e-18, \
-7.365061312846079e-18, -1.8017311599443864e-17, \
1.3632670493372523e-17, 9.691788305665686e-9, 4.993157918859164e-18, \
2.2231430136800826e-17, -1.2320025353462859e-17, \
1.4541114252365403e-6, 1.1778280765262121e-17, \
-2.4176149029033655e-7, 1.3809685160527148e-8, \
-2.3656816857767396e-7, 2.1865709723710056e-17, \
1.4098201214799795e-17, 1.7533897165166863e-17, \
-2.403370941353798e-7, 1.547953707557564e-17, 4.055693177268606e-8, \
-2.330812993763662e-9, 1.4787186135928427e-18, \
-4.256283588301072e-12, -1.6962967452121781e-9, \
1.3468706088056037e-10, 7.039294301406214e-18, \
2.4921685316853078e-11, 5.3407938818735074e-18, \
-1.5451582512948192e-17, 1.9860490710405045e-17, \
-4.256297694476767e-12, -2.6508474480020966e-12, \
4.957381921581916e-13, 9.998581554886732e-9, -1.557055906168613e-17, \
-1.69629677367044e-9, 9.781790425263753e-11, -2.2145010208905382e-17, \
5.521895217491416e-13, 9.480602559285577e-11, -7.845387360851316e-12, \
4.877027615390977e-19, -8.304944942380587e-8, 1.2029966213513953e-17, \
3.681016273125459e-18, 8.633597372190237e-18, 1.3809685143645525e-8, \
2.4921692388861975e-11, -7.930829190471786e-10, \
-3.1300867387956224e-18, 1.3508407614788492e-8, \
2.1112633587367902e-17, 1.093830999635661e-17, \
-1.4223394710281177e-17, -9.524567597519616e-18, \
2.172853304064994e-18, 2.122050347700959e-17, 2.674813870399168e-18, \
1.3809685145902462e-8, -7.67212963631507e-18, -9.994221852161832e-18, \
-1.4065250265456922e-17, -2.33081297795058e-9, \
-4.256317130542103e-12, 1.3468704424888862e-10, \
2.4921706600837316e-11, -1.6919485123646477e-17, \
-4.256295141429551e-12, 4.957468816941924e-13, \
-1.0403872172976944e-17, 9.781790534544322e-11, \
5.521742275265616e-13, -7.845373400764218e-12, -8.304944942381204e-8, \
6.213877973917897e-19, 1.3809685139229967e-8, -7.930828928605235e-10, \
1.3508407603956329e-8, 2.241011046794735e-17, -4.431068455386153e-18, \
2.21337484259214e-17, 1.3809685146432093e-8, -1.8499180202967373e-17, \
-2.330812987675844e-9, 1.3468704222672626e-10, \
2.0103420071509566e-17, 4.957296247151822e-13, 9.781791730858947e-11, \
-7.84537578438076e-12, 8.005553118508224e-18, -7.930828928768546e-10, \
1.8699587787485835e-17, 9.92044101663923e-18, 7.700788475584564e-18, \
1.3468706882065665e-10, 4.957716876117788e-13, \
-7.845346406125981e-12, -7.930829192349243e-10, 2.26500035194646e-17, \
1.346870602400161e-10, -7.845370301281788e-12, 6.314198641196867e-19, \
-7.845353300442699e-12, -7.84536308227693e-12, \
1.3113621378051747e-12]